In [1]:
## Book to scrape the wiki pages of the minor college athletic conferences that once sponsered football
# Code will extract and clean the data to be added to the yearly_conference_members CSV file 

import pandas as pd
import requests
from bs4 import BeautifulSoup

# Read in the CSV file with names and links to conference wiki pages
df = pd.read_csv('..\TEMP\CONFERENCE CHECKLIST.csv')

# Global varriables for the data frame

# Columns to keep in each dataframe
keep_cols = ['Institution', 'Nickname', 'Location', 'Founded', 
           'Joined', 'Left', 'Enrollment', 'Conference']

# columns to store as integers
int_columns = ['Founded', 'Joined', 'Left', 'Enrollment']

# Empty List of dataframes to be concatenated
df_list = []

In [2]:
df.head()


# create a dictionary of the conference names and links
conf_dict = dict(zip(df['conference'], df['wiki_link']))
print (conf_dict)


{'A-10': 'https://en.wikipedia.org/wiki/Atlantic_10_Conference', 'ACC': 'https://en.wikipedia.org/wiki/Atlantic_Coast_Conference', 'ASUN': 'https://en.wikipedia.org/wiki/ASUN_Conference', 'Alabama Intercollegiate/Collegiate': 'https://en.wikipedia.org/wiki/Southern_States_Conference', 'American': 'https://en.wikipedia.org/wiki/American_Athletic_Conference', 'Atlantic 10': 'https://en.wikipedia.org/wiki/Atlantic_10_Conference#Football_(1997–2006)', 'B1G': 'https://en.wikipedia.org/wiki/Big_Ten_Conference', 'BIAA': 'https://en.wikipedia.org/wiki/Border_Intercollegiate_Athletic_Association', 'BORDER': 'https://en.wikipedia.org/wiki/Border_Conference', 'Big 12': 'https://en.wikipedia.org/wiki/Big_12_Conference', 'Big 4 Conference': 'https://en.wikipedia.org/wiki/Big_Four_Conference_(Oklahoma)', 'Big 8': 'https://en.wikipedia.org/wiki/Big_Eight_Conference', 'Big East': 'https://en.wikipedia.org/wiki/Big_East_Conference_(1979%E2%80%932013)', 'Big Sky': 'https://en.wikipedia.org/wiki/Big_Sky_

## Notes about A-10 conference from wiki

<!-- Football Only Notes -->
Further membership changes saw the league expand to its maximum of 16 members. From 1997 through 2006, the league also operated a football conference; during that period, more than 20 schools were participating in A-10 competition in at least one sport. This ended when the A-10 football programs all departed to join a new football conference sponsored by the Colonial Athletic Association (CAA; now known as the Coastal Athletic Association). In 2012, Butler joined the conference after leaving the Horizon League and VCU joined after leaving the CAA.

<!-- Former football-only members -->
After expansion in the Colonial Athletic Association brought that conference to 6 football-playing schools, it was agreed that the CAA would take over management of the Atlantic 10's football conference starting in the 2007–08 school year as the legally separate entity of CAA Football. All the schools on this list (except Boston U. and Connecticut) were in the A-10 football conference when it became CAA Football, but Hofstra and Northeastern discontinued their football programs after the 2009–10 school year. Membership dates include time in the Yankee Conference (which was an all-sports conference from the 1947–48 to 1975–76 seasons, and a football-only conference after that) which merged into the A-10 in the 1997–98 school year.

In [3]:
# ## A-10 Conference
## NOTES:
### Footbal conference operated from 1997-2006

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['A-10'])

# try to open tables with pandas
a10_table = pd.read_html(wiki_page.text)

current = a10_table[2]
former = a10_table[4]
football_only = a10_table[6]
former
football_only

# # concatonate all into one dataframe
frames = [current, former, football_only]
a10_conf = pd.concat(frames)

# add conference name to each row
a10_conf['Conference'] = 'A-10'


# Change one name for ease of use
# 'Virginia Polytechnic Institute and State University' to 'Virginia Tech'
a10_conf['Institution'] = a10_conf['Institution'].replace('Virginia Polytechnic Institute and State University', 'Virginia Tech')

# clean 'Joined' & 'Leave' column - some have values like '1982-83', change to simply '1982'
a10_conf['Joined'] = a10_conf['Joined'].str[:4]
a10_conf['Left'] = a10_conf['Left'].str[:4]

# Manual adjustments for schools that joined and left more than once
# drop original Pennsylvania State University
a10_conf = a10_conf[a10_conf['Institution'] != 'Pennsylvania State University']
# write new Pennsylvania State University rows manually
psu1 = {'Institution': 'Pennsylvania State University', 'Nickname': 'Nittany Lions', 'Location': 'University Park, Pennsylvania', 'Founded': 1855, 'Joined': 1982, 'Left': 1991, 'Enrollment': 46_810, 'Conference': 'A-10'}
psu2 = {'Institution': 'Pennsylvania State University', 'Nickname': 'Nittany Lions', 'Location': 'University Park, Pennsylvania', 'Founded': 1855, 'Joined': 1976, 'Left': 1979, 'Enrollment': 46_810, 'Conference': 'A-10'}

# add the new rows back to the dataframe
a10_conf = a10_conf.append(psu1, ignore_index=True)
a10_conf = a10_conf.append(psu2, ignore_index=True)


# Clean Institution column of citations
a10_conf['Institution'] = a10_conf['Institution'].str.replace('\[.*\]', '')


# drop the unneccesary columns
a10_conf = a10_conf[keep_cols]
# # make sure the columns are the correct data type
# # deal with the NaN values
# a10_conf = a10_conf.fillna(0)
# a10_conf[int_columns] = a10_conf[int_columns].astype(int)

len(a10_conf)
# a10_conf.tail(20)
# drop a ll but the neseccary columns
# a10_conf.info()

# add to the list of dataframes
df_list.append(a10_conf)


C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\2920900230.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  a10_conf = a10_conf.append(psu1, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\2920900230.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  a10_conf = a10_conf.append(psu2, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\2920900230.py:46: FutureWarning: The default value of regex will change from True to False in a future version.
  a10_conf['Institution'] = a10_conf['Institution'].str.replace('\[.*\]', '')


## ASUN Conference

The Atlantic Sun Conference (ASUN) is a collegiate athletic conference operating mostly in the Southeastern United States. The league participates at the NCAA Division I level, and began sponsoring football at the Division I FCS level in 2022. Originally established as the Trans America Athletic Conference (TAAC) in 1978,[1] it was renamed as the Atlantic Sun Conference in 2001, 

In [4]:
## ASUN Conference - Have only sponsered football conference since 2022 - so need to change all joined dates to 2022
## Might introduce more problems later, but we'll see

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['ASUN'])

# try to open tables with pandas
asun_table = pd.read_html(wiki_page.text)

current = asun_table[1]
# Change Joined column to 2022
current['Joined'] = 2022
# add empty left column
current['Left'] = ''

# Add conference columns
current['Conference'] = 'ASUN'

# dropp unneccesary columns
ASUN_df = current[keep_cols]

# add to the list of dataframes
df_list.append(ASUN_df)

# ASUN_df

# The Southern States Conference (SSC) 
was an affiliate of the National Association of Intercollegiate Athletics that included member institutions in the U.S. states of Alabama, Georgia, Mississippi, and Florida.[1][2] The league existed from 1938 to 1997.[2]

History
The league was established in December 1938 as the Alabama Intercollegiate Conference (AIC), comprising schools from just that state. The six charter members were: Jacksonville State Teachers College, Saint Bernard College, Troy State Teachers College, Snead Junior College, Livingston State Teachers College, and Marion Military Institute.[1][3] The league ceased operations in 1942 because of World War II and because several member schools dropped their intercollegiate athletics programs.[4] The AIC was reformed again in January 1948 after a five year lapse.[4] In 1959 it was renamed the Alabama Collegiate Conference (ACC),[5] and then in May 1972, the league was rebranded as Southern States Conference.[6]

At the conclusion of the 1994–95 school year, two schools left the SSC, causing league membership to dip below the six required to have an NAIA championship in every sport except basketball.[7] The SSC then existed in 1995–96 as a basketball-only conference.[7][8] It returned to an all-sports conference for 1996–97 before dissolving.[9]

In [5]:
# Southern States Conference

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['Alabama Intercollegiate/Collegiate'])

# try to open tables with pandas
ssc_table = pd.read_html(wiki_page.text)

member = ssc_table[1]
# clean Joined and Left columns of non digits
member['Joined'] = member['Joined'].str.replace('\D', '')

# add conference column
member['Conference'] = 'Southern States Conference'
# add empty Enrollment column
member['Enrollment'] = ''

# keep only the neseccesary columns
southern_states_df = member[keep_cols]

# southern_states_df

# add to list of dataframes
df_list.append(southern_states_df)




C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\406920550.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  member['Joined'] = member['Joined'].str.replace('\D', '')


# The Big Four
The Big Four Conference was an intercollegiate athletic conference that existed from 1929 to 1932. Its membership was centered on the state of Oklahoma.[1]

In [6]:
# Big - 4 Conference - Only existed for 3 years and only had 4 members - one relevent

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['Big 4 Conference'])

# try to open tables with pandas
big4_table = pd.read_html(wiki_page.text)

member = big4_table[0]

# creat eh columns to match the other dataframes

member['Enrollment'] = ''
member['Conference'] = 'Big 4 Conference'
# drop citations from Nickname column
member['Nickname'] = member['Nickname'].str.replace('\[.*\]', '')

# add values to the Joined and Left columns
member['Joined'] = 1929
member['Left'] = 1932
Big_4_df = member[keep_cols]

# add to list of dataframes
df_list.append(Big_4_df)


Big_4_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3880050761.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  member['Nickname'] = member['Nickname'].str.replace('\[.*\]', '')


,Institution,Nickname,Location,Founded,Joined,Left,Enrollment,Conference
0,Oklahoma Baptist University,Bison,"Shawnee, Oklahoma",1910,1929,1932,,Big 4 Conference
1,Oklahoma City University,Goldbugs,"Oklahoma City, Oklahoma",1904,1929,1932,,Big 4 Conference
2,Phillips University,Haymakers,"Enid, Oklahoma",1906,1929,1932,,Big 4 Conference
3,University of Tulsa,Tulsa Golden Hurricane,"Tulsa, Oklahoma",1894,1929,1932,,Big 4 Conference


# Big Sky Conference

The Big Sky Conference (BSC) is a collegiate athletic conference affiliated with the NCAA's Division I with football competing in the Football Championship Subdivision. As of 2023, full member institutions are located in the states of Arizona, California, Colorado, Idaho, Montana, Oregon, Utah, and Washington. Two affiliate members from California are football–only participants.

Starting in 1968, the conference competed at the highest level (university division) in all sports except football (college division). The sole exception was Idaho, in the university division for football through 1977 (except 1967, 1968).[8] Football moved to the new Division I-AA in 1978, which was renamed Division I Football Championship Subdivision (FCS) in 2006.

In 1974, half of the Big Sky's ten sports were dropped (baseball, skiing, swimming, golf, and tennis), leaving football, basketball, wrestling, track, and cross country.[9][10]


In [7]:
# Big Sky Conference

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['Big Sky'])

# try to open tables with pandas
big_sky_table = pd.read_html(wiki_page.text)

current = big_sky_table[1]
former = big_sky_table[3]
# apeend the two dataframes
frames = [current, former]
big_sky_conf = pd.concat(frames)

# manually create new rows fort University of Idaho which joined and left and joined again
# drop the original row
big_sky_conf = big_sky_conf[big_sky_conf['Institution'] != 'University of Idaho']
# create two new rows with the correct values (one for each time they joined)
# RANGES 1963-1996, 2018-present
idaho1 = {'Institution': 'University of Idaho', 'Nickname': 'Vandals', 'Location': 'Moscow, Idaho', 'Founded': 1889, 'Joined': 1963, 'Left': 1996, 'Enrollment': 11_926, 'Conference': 'Big Sky'}
idaho2 = {'Institution': 'University of Idaho', 'Nickname': 'Vandals', 'Location': 'Moscow, Idaho', 'Founded': 1889, 'Joined': 2018, 'Left': '', 'Enrollment': 11_926, 'Conference': 'Big Sky'}

# add the new rows back to the dataframe
big_sky_conf = big_sky_conf.append(idaho1, ignore_index=True)
big_sky_conf = big_sky_conf.append(idaho2, ignore_index=True)

# remove citations from Institution column
big_sky_conf['Institution'] = big_sky_conf['Institution'].str.replace('\[.*\]', '')

# keep only the neseccesary columns
big_sky_conf = big_sky_conf[keep_cols]

# add to list of dataframes
df_list.append(big_sky_conf)



# current
# former
# big_sky_conf

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\4189999091.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_sky_conf = big_sky_conf.append(idaho1, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\4189999091.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_sky_conf = big_sky_conf.append(idaho2, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\4189999091.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
  big_sky_conf['Institution'] = big_sky_conf['Institution'].str.replace('\[.*\]', '')


# Big South Conference

The Big South Conference is a collegiate athletic conference affiliated with the NCAA's Division I. Originally a non-football conference, the Big South began sponsoring football in 2002 as part of the Football Championship Subdivision (FCS), and began operating the Big South–OVC Football Association in partnership with the Ohio Valley Conference in 2023. The Big South, founded in 1983, is firmly rooted in the South Atlantic region of the United States, with full member institutions located in North Carolina, South Carolina, and Virginia. Associate members are located in Georgia, Pennsylvania, Rhode Island, and South Carolina.[1]

In [8]:
# Big South - has current and former members
# Football only sponsered since 2002

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['Big South'])

# try to open tables with pandas
big_south_table = pd.read_html(wiki_page.text)

current = big_south_table[1]
former = big_south_table[3]

# append to single dataframe
frames = [current, former]
big_south_conf = pd.concat(frames)

# create two rows manually for Campbell University which joined and left and joined again
# drop the original row
big_south_conf = big_south_conf[big_south_conf['Institution'] != 'Campbell University']
# create two new rows with the correct values (one for each time they joined)
# RANGES 1994-2000, 2008-present
campbell1 = {'Institution': 'Campbell University', 'Nickname': 'Fighting Camels', 'Location': 'Buies Creek, North Carolina', 'Founded': 1887, 'Joined': 1983, 'Left': 1994, 'Enrollment': 6_448, 'Conference': 'Big South'}
campbell2 = {'Institution': 'Campbell University', 'Nickname': 'Fighting Camels', 'Location': 'Buies Creek, North Carolina', 'Founded': 1887, 'Joined': 1994, 'Left': '2023', 'Enrollment': 6_448, 'Conference': 'Big South'}

# add new rows to the dataframe
big_south_conf = big_south_conf.append(campbell1, ignore_index=True)

# manually fix Liberty University which joined - overwirte with 1991
big_south_conf.loc[big_south_conf['Institution'] == 'Liberty University', 'Joined'] = 1991
# Remove citations from joined and left columns
# big_south_conf['Joined'] = big_south_conf['Joined'].str.replace('\D', '')
# big_south_conf['Left'] = big_south_conf['Left'].str.replace('\D', '')



# add the new rows back to the dataframe
big_south_conf = big_south_conf.append(campbell1, ignore_index=True)

# Add conference column
big_south_conf['Conference'] = 'Big South'

# Keep only the neseccesary columns
big_south_conf = big_south_conf[keep_cols]

# add to list of dataframes
df_list.append(big_south_conf)


# current
# # former
# big_south_conf


C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\1023988458.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_south_conf = big_south_conf.append(campbell1, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\1023988458.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_south_conf = big_south_conf.append(campbell1, ignore_index=True)


# Big West Conference

The Big West Conference (BWC) is an American collegiate athletic conference whose member institutions participate in the National Collegiate Athletic Association's Division I. The conference was originally formed on July 1, 1969, as the Pacific Coast Athletic Association (PCAA), and in 1988 was renamed the Big West Conference. The conference stopped sponsoring college football after the 2000 season.

Among the conference's 11 member institutions, 10 are located in California (with 9 located in Southern California alone) and one is located in Hawaii. All of the schools are public universities, with the California schools evenly split between the California State University and the University of California systems. In addition, one affiliate member plays two sports in the BWC not sponsored by its home conference.

In [9]:
# Big West Conference - stopped sponsering football in 2000 so will need to change all joined dates to 2000

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['Big West'])

# try to open tables with pandas
big_west_table = pd.read_html(wiki_page.text)

current = big_west_table[1]
former = big_west_table[3]
football_only = big_west_table[5]

# cleanup joined and left dates in the football only dataframe - keep first four characters
football_only['Joined'] = football_only['Joined'].str[:4]
football_only['Left'] = football_only['Left'].str[:4]

# append them all to one dataframe
frames = [current, former, football_only]
big_west_conf = pd.concat(frames)

# add conference column
big_west_conf['Conference'] = 'Big West'



# clean joined column of any citations
big_west_conf['Joined'] = big_west_conf['Joined'].str.replace('\[.*\]', '')

# fix University of California, Santa Barbara(UC Santa Barbara) Manually
# drop the original row
big_west_conf = big_west_conf[big_west_conf['Institution'] != 'University of California, Santa Barbara(UC Santa Barbara)']



# San Diego State University	& University of the Pacific	
# keep first four chartacters from the Joined & Left column
big_west_conf['Joined'] = big_west_conf['Joined'].str[:4]
big_west_conf['Left'] = big_west_conf['Left'].str[:4]

# if any Left date are >2000 or Nan change to 2000
big_west_conf.loc[big_west_conf['Left'] > '2000', 'Left'] = 2000
big_west_conf['Left'] = big_west_conf['Left'].fillna(2000)

# creat two new rows with correct join and left dates
ucsb1 = {'Institution': 'University of California, Santa Barbara', 'Nickname': 'Gauchos', 'Location': 'Santa Barbara, California', 'Founded': 1909, 'Joined': 1969, 'Left': 1973, 'Enrollment': 25_057, 'Conference': 'Big West'}
ucsb2 = {'Institution': 'University of California, Santa Barbara', 'Nickname': 'Gauchos', 'Location': 'Santa Barbara, California', 'Founded': 1909, 'Joined': 1976, 'Left': 2000, 'Enrollment': 25_057, 'Conference': 'Big West'}

# add back to DF
big_west_conf = big_west_conf.append(ucsb1, ignore_index=True)
big_west_conf = big_west_conf.append(ucsb2, ignore_index=True)


# keep only the neseccesary columns
big_west_conf = big_west_conf[keep_cols]

# add to list of dataframes
df_list.append(big_west_conf)

current
former
football_only
big_west_conf


C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3132467921.py:27: FutureWarning: The default value of regex will change from True to False in a future version.
  big_west_conf['Joined'] = big_west_conf['Joined'].str.replace('\[.*\]', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3132467921.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_west_conf = big_west_conf.append(ucsb1, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3132467921.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_west_conf = big_west_conf.append(ucsb2, ignore_index=True)


,Institution,Nickname,Location,Founded,Joined,Left,Enrollment,Conference
0,California Polytechnic State University(Cal Po...,Mustangs,"San Luis Obispo, California",1901,1996,2000,22287,Big West
1,"California State University, Bakersfield(Cal S...",Roadrunners,"Bakersfield, California",1965,2020,2000,9261,Big West
2,"California State University, Fullerton(Cal Sta...",Titans,"Fullerton, California",1957,1974,2000,38726,Big West
3,"California State University, Northridge(Cal St...",Matadors,"Northridge, California[c]",1958,2001,2000,38511,Big West
4,University of Hawaiʻi at Mānoa(Hawaiʻi),Rainbow Warriors& Rainbow Wahine[e],"Honolulu, Hawaii",1907,2012,2000,19074,Big West
5,"California State University, Long Beach(Long B...",Beach[f],"Long Beach, California",1949,1969,2000,39360,Big West
6,"University of California, Davis(UC Davis)",Aggies,"Davis, California",1905,2007,2000,40772,Big West
7,"University of California, Irvine(UC Irvine)",Anteaters,"Irvine, California",1965,1977,2000,37243,Big West
8,"University of California, Riverside(UC Riverside)",Highlanders,"Riverside, California",1954,2001,2000,26809,Big West
9,"University of California, San Diego(UC San Die...",Tritons,"La Jolla, California[h]",1960,2020,2000,42968,Big West


# Colonial  / Coastal Conference Athletic Association

The Coastal Athletic Association Football Conference, formerly the Colonial Athletic Association Football Conference, branded as CAA Football, is a collegiate athletic conference affiliated with the NCAA's Division I whose full members are located in East Coast states, from Maine to North Carolina. Most of its members are public universities, and the conference is headquartered in Richmond, Virginia. The conference is run by the same administration as the multisport conference Coastal Athletic Association (CAA; formerly the Colonial Athletic Association) but is legally a different entity.[1]

## History
CAA Football was formed in 2005, although it did not begin play until 2007, as a separate conference independent of the CAA, but administered by the CAA front office. In the 2004–05 academic year, the CAA had five member schools that sponsored football, all of them as football-only members of the Atlantic 10 Conference. In 2005, Northeastern accepted the CAA's offer of membership, giving the CAA the six football-playing members it needed under NCAA rules to organize a football conference. At that time, the CAA announced it would launch its new football conference in 2007. Next, the CAA invited the University of Richmond to become a football-only member effective in 2007. Once UR accepted the offer, this left the A10 football conference with only five members, less than the six required under NCAA rules. As a result, the remaining A10 football programs all decided to join the CAA for football only, ending A10 football. Since the CAA football conference had the same members as the A10 the previous year, it can be said that the CAA football conference is the A10 football conference under new management.

The CAA football conference's earliest roots are in the New England Conference, founded in 1938 by four state-supported universities in that region plus Northeastern; three of the public schools are currently in CAA Football. However, neither the multi-sports CAA nor CAA Football includes the New England Conference in CAA Football history.[2] After the departure of Northeastern in 1945, the remaining members joined New England's other land-grant colleges, Massachusetts State College (now the University of Massachusetts Amherst) and the University of Vermont, to form the Yankee Conference under a new charter in 1946, with competition starting in 1947. That conference eventually dropped all sports other than football in 1975. Starting in the 1980s, it expanded to include many schools outside its original New England base. After the NCAA voted to limit the influence of single-sport conferences, the Yankee merged with the A-10 in 1997.

See also: 2010–2013 Colonial Athletic Association realignment
CAA Football went through many changes during the early 2010s with the loss of Georgia State, Massachusetts, and Old Dominion and the addition of Albany, Elon, and Stony Brook. Stability was maintained for a decade before the departure of James Madison in 2021 leading to the addition of Campbell, Hampton, Monmouth, North Carolina A&T, and Bryant from 2022 to 2024.

In [10]:
# Coastal Athletic Association / Colonial Athletic Association

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['CAA'])

# try to open tables with pandas
caa_table = pd.read_html(wiki_page.text)

current = caa_table[2]
former = caa_table[4]

# combine into single dataframe
caa_df = pd.concat([current, former])

# Add conference column
caa_df['Conference'] = 'CAA'

# keep only the neseccesary columns
caa_df = caa_df[keep_cols]

# add to list of dataframes
df_list.append(caa_df)

current
# former
# caa_df

,Institution,Location,Founded,Joined,Type,Enrollment,Nickname,Colors,PrimaryConference
0,University at Albany,"Albany, New York",1844,2013,Public,17944,Great Danes,.mw-parser-output .legend{page-break-inside:av...,AmEast
1,Campbell University,"Buies Creek, North Carolina",1887,2023,Private,5622,Fighting Camels,NaN,CAA
2,University of Delaware,"Newark, Delaware",1743,2007,Public,23281,Fightin' Blue Hens,NaN,CAA
3,Elon University,"Elon, North Carolina",1889,2014,Private,6991,Phoenix,NaN,CAA
4,Hampton University,"Hampton, Virginia",1868,2022,Private(HBCU),3516,Pirates,NaN,CAA
5,University of Maine,"Orono, Maine",1865,2007,Public,11404,Black Bears,NaN,AmEast
6,Monmouth University,"West Long Branch, New Jersey",1933,2022,Private,5675,Hawks,NaN,CAA
7,University of New Hampshire,"Durham, New Hampshire",1866,2007,Public,15305,Wildcats,NaN,AmEast
8,North Carolina A&T State University,"Greensboro, North Carolina",1891,2023,Public(HBCU),13332,Aggies,NaN,CAA
9,Towson University,"Towson, Maryland",1866,2007,Public,22923,Tigers,NaN,CAA


# California Collegiate Athletic Association

The California Collegiate Athletic Association (CCAA) is a college athletic conference affiliated with the National Collegiate Athletic Association (NCAA) at the Division II level.[1] All of its current members are public universities, and upon UC San Diego's departure on July 1, 2020, all are members of the California State University system (two of them being Cal Polys).[2]

It was founded in December 1938 and began competition in 1939. The commissioner of the CCAA is Allen Hardison. CCAA offices are located in Aliso Viejo, California.[3] The CCAA is the most successful conference in NCAA Division II, as its former and current members have won 155 National Championships.[4]

In [11]:
## # California Collegiate Athletic Association

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['CCAA'])

# try to open tables with pandas
ccaa_table = pd.read_html(wiki_page.text)

current = ccaa_table[1]
former = ccaa_table[2]

# append to single dataframe
ccaa_df = pd.concat([current, former])


# rename Location[a] to Location
ccaa_df = ccaa_df.rename(columns={'Location[a]': 'Location'})

# create two rows for California State - Los Angeles which joined and left and joined again
#  Cal State LA left the CCAA after the 1968–69 school year to join the Pacific Coast Athletic Association (PCAA; now known as the Big West Conference), but re-joined back to the CCAA, effective in the 1974–75 school year
# drop the original row
ccaa_df = ccaa_df[ccaa_df['Institution'] != 'California State University, Los Angeles']
# create two new rows with the correct values (one for each time they joined)
# RANGES 1963-1968, 1974-present
cal_state_la1 = {'Institution': 'California State - Los Angeles', 'Nickname': 'Golden Eagles', 'Location': 'Los Angeles, California', 'Founded': 1947, 'Joined': 1963, 'Left': 1968, 'Enrollment': 27_959, 'Conference': 'CCAA'}
cal_state_la2 = {'Institution': 'California State - Los Angeles', 'Nickname': 'Golden Eagles', 'Location': 'Los Angeles, California', 'Founded': 1947, 'Joined': 1974, 'Left': '', 'Enrollment': 27_959, 'Conference': 'CCAA'}

# add the new rows back to the dataframe
ccaa_df = ccaa_df.append(cal_state_la1, ignore_index=True)
ccaa_df = ccaa_df.append(cal_state_la2, ignore_index=True)



# Standardize the Institution column - rename the rows with California State University, _______ to California State - _______
ccaa_df['Institution'] = ccaa_df['Institution'].str.replace('California State University, ', 'California State - ')

# Add conference column
ccaa_df['Conference'] = 'CCAA'

# keep only the neseccesary columns
ccaa_df = ccaa_df[keep_cols]

# add to list of dataframes
df_list.append(ccaa_df)


# current
# former

# ccaa_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\305976963.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ccaa_df = ccaa_df.append(cal_state_la1, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\305976963.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ccaa_df = ccaa_df.append(cal_state_la2, ignore_index=True)


# Conference Carolinas / Carolinas-Virginia Athletic Conference (CVAC) or the Carolinas Conference

### may not have any relevant teams

Conference Carolinas, formerly known as the Carolinas-Virginia Athletic Conference (CVAC) or the Carolinas Conference, is a college athletic conference affiliated with the National Collegiate Athletic Association (NCAA) primarily at the Division II level. It is also considered as one of the seven Division I conferences for men's volleyball. Originally formed in 1930, the league reached its modern incarnation in 1994. Member institutions are located in the southeastern United States in the states of Georgia, North Carolina, South Carolina, and Tennessee. The Conference Carolinas membership currently consists of 14 small colleges or universities, 12 private and two public.

In [12]:
## # Conference Carolinas / Carolinas-Virginia Athletic Conference (CVAC) or the Carolinas Conference

# call the wiki page using dictionary
# Carolinas
wiki_page = requests.get(conf_dict['Carolinas'])

# try to open tables with pandas
carolinas_table = pd.read_html(wiki_page.text)

current = carolinas_table[1]
former = carolinas_table[4]

# append to single dataframe
carolinas_df = pd.concat([current, former])

# Add conference column
carolinas_df['Conference'] = 'Carolinas Conference'

# drop rows that are not relevent
# Lenoir–Rhyne College[g][h]
carolinas_df = carolinas_df[carolinas_df['Institution'] != 'Lenoir–Rhyne College[g][h]']


# add correct Joined date for University of North Carolina at Pembroke (1976)
carolinas_df.loc[carolinas_df['Institution'] == 'University of North Carolina at Pembroke', 'Joined'] = 1976

# Clean the Joined and left column by dropping citations and keeping only the first four characters
carolinas_df['Joined'] = carolinas_df['Joined'].str.replace('\D', '')
carolinas_df['Left'] = carolinas_df['Left'].str.replace('\D', '')

# keep only relevent columns
carolinas_df = carolinas_df[keep_cols]

# add to list of dataframes
df_list.append(carolinas_df)




# current
# former
# carolinas_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3699212365.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
  carolinas_df['Joined'] = carolinas_df['Joined'].str.replace('\D', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3699212365.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  carolinas_df['Left'] = carolinas_df['Left'].str.replace('\D', '')


# Dixie Conference

The Dixie Conference was the name of two collegiate athletic leagues in the United States The first operated from 1930 until the United States' entry into World War II in 1942. The second conference to use the name existed from 1948 to 1954.

## Original Conf
At the Southern Intercollegiate Athletic Association (SIAA) annual convention in 1930, nine of the association's members announced the formation of the Dixie Conference to facilitate scheduling of games among the group.[1] The charter members were Birmingham-Southern College, Howard College (now Samford University), Southwestern of Memphis (now Rhodes College), Centre College, University of Chattanooga (now the University of Tennessee at Chattanooga), Spring Hill College and Mercer University;[1] Loyola University New Orleans joined the Dixie two years later.[2]

## Reformed Conference
Formation of purely amateur conference
In 1948, the administration of Florida State University, which had returned to coeducation in 1947 after more than 40 years as a women's college, wrote leaders at other southern institutions seeking to create a "purely amateur" athletic conference.[22] The NCAA had recently tightened its rules on the amount of money that could be paid to collegiate athletes, which caused difficulty for many established athletic programs.[23] Florida State hoped to create a new conference based on the principle of complete amateurism, including no athletic scholarships.[22]

The original Dixie Conference lineup included, in addition to Florida State, Howard College (now Samford University) from Alabama, Stetson University and the University of Tampa from Florida, Lambuth College from Tennessee, Mercer University and Oglethorpe University from Georgia, and Millsaps College and Mississippi College from Mississippi.[24] Of the nine charter members, six (FSU, Howard, Millsaps, Mississippi, Stetson, and Tampa) played football.

Membership transitions
In 1949, Lambuth, Stetson and Tampa left the conference, while Florida Southern College joined.[25] In December 1950, after winning the first three Dixie football titles without losing a single conference game, founding member Florida State withdrew to become an independent and began offering scholarships.[26][27]

In [13]:
## Dixie Conference

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['Dixie'])

# Need to create a table from scratch because the wiki page is not in a table format
# create empty table
dixie_df = pd.DataFrame(columns=['Institution', 'Nickname', 'Location', 'Founded', 
           'Joined', 'Left', 'Enrollment', 'Conference'])

# Original Members (1930-1942)
#  Birmingham-Southern College, Howard College (now Samford University), Southwestern of Memphis (now Rhodes College),
#  Centre College, University of Chattanooga (now the University of Tennessee at Chattanooga), Spring Hill College and Mercer University;
# Joined in 1932 - Loyola University New Orleans 

# create a row for each member of the original conference
# Birmingham-Southern College
# Samford University
# Rhodes College
# Tennessee - Chattanooga
# Spring Hill College
# Mercer University
# Above all joined in 1930 and left in 1942
# Loyola University New Orleans joined in 1932 and left in 1942

# create rows
birmingham_southern = {'Institution': 'Birmingham-Southern College', 'Nickname': 'Panthers', 'Location': 'Birmingham, Alabama', 'Founded': 1856, 'Joined': 1930, 'Left': 1942, 'Enrollment': 1_300, 'Conference': 'Dixie'}
samford = {'Institution': 'Samford University', 'Nickname': 'Bulldogs', 'Location': 'Homewood, Alabama', 'Founded': 1841, 'Joined': 1930, 'Left': 1942, 'Enrollment': 5_692, 'Conference': 'Dixie'}
rhodes = {'Institution': 'Rhodes College', 'Nickname': 'Lynx', 'Location': 'Memphis, Tennessee', 'Founded': 1848, 'Joined': 1930, 'Left': 1942, 'Enrollment': 2_000, 'Conference': 'Dixie'}
tennessee_chattanooga = {'Institution': 'University of Tennessee at Chattanooga', 'Nickname': 'Mocs', 'Location': 'Chattanooga, Tennessee', 'Founded': 1886, 'Joined': 1930, 'Left': 1942, 'Enrollment': 11_561, 'Conference': 'Dixie'}
spring_hill = {'Institution': 'Spring Hill College', 'Nickname': 'Badgers', 'Location': 'Mobile, Alabama', 'Founded': 1830, 'Joined': 1930, 'Left': 1942, 'Enrollment': 1_500, 'Conference': 'Dixie'}
mercer = {'Institution': 'Mercer University', 'Nickname': 'Bears', 'Location': 'Macon, Georgia', 'Founded': 1833, 'Joined': 1930, 'Left': 1942, 'Enrollment': 8_600, 'Conference': 'Dixie'}
loyola = {'Institution': 'Loyola University New Orleans', 'Nickname': 'Wolf Pack', 'Location': 'New Orleans, Louisiana', 'Founded': 1912, 'Joined': 1932, 'Left': 1942, 'Enrollment': 4_261, 'Conference': 'Dixie'}

# create list of rows
dixie_rows = [birmingham_southern, samford, rhodes, tennessee_chattanooga, spring_hill, mercer, loyola]

# add rows to dataframe
dixie_df = dixie_df.append(dixie_rows, ignore_index=True)

# Create the rows for the reformer conference   





# Reforme3d Conference Members
# The original Dixie Conference lineup included, in addition to Florida State, Howard College (now Samford University) from Alabama, Stetson University and the University of Tampa from Florida, Lambuth College from Tennessee, Mercer University and Oglethorpe University from Georgia, and Millsaps College and Mississippi College from Mississippi.[24] Of the nine charter members, six (FSU, Howard, Millsaps, Mississippi, Stetson, and Tampa) played football.

# Florida State University - Joined in 1948 and left in 1950
# Howard College (now Samford University) - Joined in 1948 and left in 1949
# Stetson University - Joined in 1948 and left in 1949
# University of Tampa - Joined in 1948 and left in 1949
# Florida Southern - Joine din 1949 Left 1954
# Lambuth College - Joined in 1948 and left in 1949
# Mercer University - Joined in 1948 and left in 1954
# Oglethorpe University - Joined in 1948 and left in 1954
# Millsaps College - Joined in 1948 and left in 1954
# Mississippi College - PRIVATE SCHOOL NOT TO BE CONFUSED WITH OLE MISS - LEAVING OUT OF DF TO AVOID CONFUSTION- Joined in 1948 and left in 1954

# create rows
florida_state = {'Institution': 'Florida State University', 'Nickname': 'Seminoles', 'Location': 'Tallahassee, Florida', 'Founded': 1851, 'Joined': 1948, 'Left': 1950, 'Enrollment': 41_867, 'Conference': 'Dixie'}
howard = {'Institution': 'Howard College', 'Nickname': 'Bulldogs', 'Location': 'Homewood, Alabama', 'Founded': 1841, 'Joined': 1948, 'Left': 1949, 'Enrollment': 5_692, 'Conference': 'Dixie'}
stetson = {'Institution': 'Stetson University', 'Nickname': 'Hatters', 'Location': 'DeLand, Florida', 'Founded': 1883, 'Joined': 1948, 'Left': 1949, 'Enrollment': 4_378, 'Conference': 'Dixie'}
tampa = {'Institution': 'University of Tampa', 'Nickname': 'Spartans', 'Location': 'Tampa, Florida', 'Founded': 1931, 'Joined': 1948, 'Left': 1949, 'Enrollment': 9_269, 'Conference': 'Dixie'}
florida_southern = {'Institution': 'Florida Southern College', 'Nickname': 'Moccasins', 'Location': 'Lakeland, Florida', 'Founded': 1883, 'Joined': 1949, 'Left': 1954, 'Enrollment': 3_000, 'Conference': 'Dixie'}
lambuth = {'Institution': 'Lambuth College', 'Nickname': 'Eagles', 'Location': 'Jackson, Tennessee', 'Founded': 1843, 'Joined': 1948, 'Left': 1949, 'Enrollment': 1_000, 'Conference': 'Dixie'}
mercer = {'Institution': 'Mercer University', 'Nickname': 'Bears', 'Location': 'Macon, Georgia', 'Founded': 1833, 'Joined': 1948, 'Left': 1954, 'Enrollment': 8_600, 'Conference': 'Dixie'}
oglethorpe = {'Institution': 'Oglethorpe University', 'Nickname': 'Stormy Petrels', 'Location': 'Brookhaven, Georgia', 'Founded': 1835, 'Joined': 1948, 'Left': 1954, 'Enrollment': 1_200, 'Conference': 'Dixie'}
millsaps = {'Institution': 'Millsaps College', 'Nickname': 'Majors', 'Location': 'Jackson, Mississippi', 'Founded': 1890, 'Joined': 1948, 'Left': 1954, 'Enrollment': 1_100, 'Conference': 'Dixie'}


# make list of new rows
dixie_rows_2 = [florida_state, howard, stetson, tampa, florida_southern, lambuth, mercer, oglethorpe, millsaps]

# add to dataframe
dixie_df = dixie_df.append(dixie_rows_2, ignore_index=True)

# add to list of dataframes
df_list.append(dixie_df)

# dixie_df





C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\1922664631.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dixie_df = dixie_df.append(dixie_rows, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\1922664631.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dixie_df = dixie_df.append(dixie_rows_2, ignore_index=True)


# Gulf Coast Conference

The Gulf Coast Conference (GCC) was a short-lived NCAA college athletic conference composed of universities in the U.S. state of Texas from 1949 until 1957. The charter members of the conference were University of Houston, Midwestern University (now Midwestern State University), North Texas State College (now the University of North Texas), and Trinity University.[1] The Gulf Coast Conference spawned from then members of the Lone Star Conference, and its president was D.L. Ligon.[2] In 1956, when the NCAA created divisions, all members of the conference at the time were classified as part of the NCAA's College Division, which was later subdivided into Division II and Division III in 1973. Charter member Houston had already left for the Missouri Valley Conference by the end of 1950, and was classified as a University Division school, which later became known as Division I.

In [14]:
## Gulf Coast COnference - need to manually create because not tables to scrape

# Members - Houston, Midwestern State, North Texas, Trinity, Abilene Christian, Hardin-Simmons
# Joined - 19949, 1949, 1949, 1949, 1954, 1956
# Left - 1950, 1957, 1957, 1957, 1957, 1957
# leave enrollment and founded blank
# Conference = Gulf Coast Conference for all

# create rows
houston = {'Institution': 'University of Houston', 'Nickname': 'Cougars', 'Location': 'Houston, Texas', 'Founded': '', 'Joined': 1949, 'Left': 1950, 'Enrollment': '', 'Conference': 'Gulf Coast Conference'}
midwestern_state = {'Institution': 'Midwestern State University', 'Nickname': 'Mustangs', 'Location': 'Wichita Falls, Texas', 'Founded': '', 'Joined': 1949, 'Left': 1957, 'Enrollment': '', 'Conference': 'Gulf Coast Conference'}
north_texas = {'Institution': 'University of North Texas', 'Nickname': 'Mean Green', 'Location': 'Denton, Texas', 'Founded': '', 'Joined': 1949, 'Left': 1957, 'Enrollment': '', 'Conference': 'Gulf Coast Conference'}
trinity = {'Institution': 'Trinity University', 'Nickname': 'Tigers', 'Location': 'San Antonio, Texas', 'Founded': '', 'Joined': 1949, 'Left': 1957, 'Enrollment': '', 'Conference': 'Gulf Coast Conference'}
abilene_christian = {'Institution': 'Abilene Christian University', 'Nickname': 'Wildcats', 'Location': 'Abilene, Texas', 'Founded': '', 'Joined': 1954, 'Left': 1957, 'Enrollment': '', 'Conference': 'Gulf Coast Conference'}
hardin_simmons = {'Institution': 'Hardin-Simmons University', 'Nickname': 'Cowboys', 'Location': 'Abilene, Texas', 'Founded': '', 'Joined': 1956, 'Left': 1957, 'Enrollment': '', 'Conference': 'Gulf Coast Conference'}

# create blank dataframe
gulf_coast_df = pd.DataFrame(columns=['Institution', 'Nickname', 'Location', 'Founded', 
           'Joined', 'Left', 'Enrollment', 'Conference'])

# list of rows
gulf_coast_rows = [houston, midwestern_state, north_texas, trinity, abilene_christian, hardin_simmons]

# ADD ROWS TO DATAFRAME
gulf_coast_df = gulf_coast_df.append(gulf_coast_rows, ignore_index=True)

# add to list of dataframes
df_list.append(gulf_coast_df)

# gulf_coast_df



C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\1865711346.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gulf_coast_df = gulf_coast_df.append(gulf_coast_rows, ignore_index=True)


# Mid-South Gulf South Conference

The Gulf South Conference (GSC) is a college athletic conference affiliated with the National Collegiate Athletic Association (NCAA) at the Division II level, which operates in the Southeastern United States.

History
Originally known as the Mid-South Athletic Conference or Mid-South Conference, the Gulf South Conference was formed by six universities in the summer of 1970: Delta State, Florence State (now North Alabama), Jacksonville State, Livingston (now West Alabama), Tennessee–Martin, and Troy State (now Troy). Scheduling problems for the 1970–71 academic year limited the league to football, won by Jacksonville State.

In [15]:
# Mid- South Conference - only interested in the former members because all current members are Division II and not in other data sets

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['Gulf South'])

# try to open tables with pandas
gulf_south_table = pd.read_html(wiki_page.text)

former = gulf_south_table[5]

mid_south_df = former
# add conference column
mid_south_df['Conference'] = 'Mid-South Conference'
# keep only the neseccesary columns
mid_south_df = mid_south_df[keep_cols]
# add to list of dataframes
df_list.append(mid_south_df)

former
# mid_south_df


,Institution,Location,Founded,Affiliation,Enrollment,Nickname,Joined,Left,Currentconference,Conference
0,Arkansas Tech University,"Russellville, Arkansas",1909,Public,12009,Wonder Boys &Golden Suns,1995,2011,Great American (GAC),Mid-South Conference
1,University of Arkansas at Monticello,"Monticello, Arkansas",1910,Public,3659,Boll Weevils &Cotton Blossoms,1995,2011,Great American (GAC),Mid-South Conference
2,University of Central Arkansas,"Conway, Arkansas",1907,Public,10869,Bears &Sugar Bears,1993,2006,Atlantic Sun (ASUN)[a],Mid-South Conference
3,Harding University[b],"Searcy, Arkansas",1924,Churchesof Christ,6009,Bisons &Lady Bisons,2000,2011,Great American (GAC),Mid-South Conference
4,Henderson State University,"Arkadelphia, Arkansas",1890,Public,3530,Reddies,1993,2011,Great American (GAC),Mid-South Conference
5,Jacksonville State University,"Jacksonville, Alabama",1883,Public,9238,Gamecocks,1970,1993,Conference USA (C-USA)[a],Mid-South Conference
6,Lincoln Memorial University,"Harrogate, Tennessee",1897,Nonsectarian,4867,Railsplitters,1992,2006,South Atlantic (SAC),Mid-South Conference
7,Mississippi University for Women,"Columbus, Mississippi",1884,Public,2479,Blues[c],1993,2003,St. Louis (SLIAC)[d],Mid-South Conference
8,University of New Orleans[e],"New Orleans, Louisiana",1958,Public,8511,Privateers,2011,2012,Southland[a],Mid-South Conference
9,Nicholls State University,"Thibodaux, Louisiana",1948,Public,6366,Colonels,1971,1979,Southland[a],Mid-South Conference


# Gulf Star Conference
## Very short lived but with some relevant teams in it's history

The Gulf Star Conference was an NCAA Division II[1] conference that existed from 1984–85 to 1986–87, three academic years. All of the schools subsequently joined the Southland Conference. Dave Waples was the only Commissioner, with the Conference office located in Lake Charles, Louisiana.

In [16]:
# Gulf Star Conference

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['Gulf Star'])

# try to open tables with pandas
gulf_star_table = pd.read_html(wiki_page.text)

gulf_star_df = gulf_star_table[1]

# add conference column
gulf_star_df['Conference'] = 'Gulf Star Conference'
# add blank Enrollment column
gulf_star_df['Enrollment'] = ''
# keep only the neseccesary columns
gulf_star_df = gulf_star_df[keep_cols]
# add to list of dataframes
df_list.append(gulf_star_df)

# gulf_star_df

# https://en.wikipedia.org/wiki/Western_States_Football_League
The Western States Football League (WSFL) is a defunct American junior college football league for schools in the states of Arizona, Idaho, New Mexico and Utah that existed from 1985 to 2018. The league was part of the National Junior College Athletic Association.

In [17]:
# Western states conference - Skipping Not Relevant

# Indiana Collegiate Conference
## May have info relevant to FCS Schools

The Indiana Collegiate Conference (ICC) was a men's college athletic conference in the United States, in existence from 1950 to 1978. It consisted solely of schools in Indiana.

The charter members of the conference were Indiana State University, Butler University, Valparaiso University, the University of Evansville, Ball State University, and Saint Joseph's College (Indiana), joined in 1953 by DePauw University.

In [18]:
# Indiana Collegiate Conference

# Member schools
# Ball State (1950–1968)
# Butler (1950–1978)
# Evansville (1950–1978)
# Indiana State (1950–1968)
# Saint Joseph's (Ind.) (1950–1978)
# Valparaiso (1950–1978)
# DePauw (1953–1977)
# Indiana Central (1970–1978)
# Wabash (1970–1976)

# create rows
ball_state = {'Institution': 'Ball State University', 'Nickname': 'Cardinals', 'Location': 'Muncie, Indiana', 'Founded': 1918, 'Joined': 1950, 'Left': 1968, 'Enrollment': 22_513, 'Conference': 'Indiana Collegiate Conference'}
butler = {'Institution': 'Butler University', 'Nickname': 'Bulldogs', 'Location': 'Indianapolis, Indiana', 'Founded': 1855, 'Joined': 1950, 'Left': 1978, 'Enrollment': 4_685, 'Conference': 'Indiana Collegiate Conference'}
evansville = {'Institution': 'University of Evansville', 'Nickname': 'Purple Aces', 'Location': 'Evansville, Indiana', 'Founded': 1854, 'Joined': 1950, 'Left': 1978, 'Enrollment': 2_483, 'Conference': 'Indiana Collegiate Conference'}
indiana_state = {'Institution': 'Indiana State University', 'Nickname': 'Sycamores', 'Location': 'Terre Haute, Indiana', 'Founded': 1865, 'Joined': 1950, 'Left': 1968, 'Enrollment': 13_044, 'Conference': 'Indiana Collegiate Conference'}
saint_josephs = {'Institution': 'Saint Josephs College', 'Nickname': 'Pumas', 'Location': 'Rensselaer, Indiana', 'Founded': 1889, 'Joined': 1950, 'Left': 1978, 'Enrollment': 1_100, 'Conference': 'Indiana Collegiate Conference'}
valparaiso = {'Institution': 'Valparaiso University', 'Nickname': 'Crusaders', 'Location': 'Valparaiso, Indiana', 'Founded': 1859, 'Joined': 1950, 'Left': 1978, 'Enrollment': 4_500, 'Conference': 'Indiana Collegiate Conference'}
depauw = {'Institution': 'DePauw University', 'Nickname': 'Tigers', 'Location': 'Greencastle, Indiana', 'Founded': 1837, 'Joined': 1953, 'Left': 1977, 'Enrollment': 2_300, 'Conference': 'Indiana Collegiate Conference'}
indiana_central = {'Institution': 'University of Indianapolis', 'Nickname': 'Greyhounds', 'Location': 'Indianapolis, Indiana', 'Founded': 1902, 'Joined': 1970, 'Left': 1978, 'Enrollment': 5_500, 'Conference': 'Indiana Collegiate Conference'}
wabash = {'Institution': 'Wabash College', 'Nickname': 'Little Giants', 'Location': 'Crawfordsville, Indiana', 'Founded': 1832, 'Joined': 1970, 'Left': 1976, 'Enrollment': 900, 'Conference': 'Indiana Collegiate Conference'}

# create blank dataframe
indiana_collegiate_df = pd.DataFrame(columns=['Institution', 'Nickname', 'Location', 'Founded', 
           'Joined', 'Left', 'Enrollment', 'Conference'])

# list of rows
indiana_collegiate_rows = [ball_state, butler, evansville, indiana_state, saint_josephs, valparaiso, depauw, indiana_central, wabash]

# ADD ROWS TO DATAFRAME
indiana_collegiate_df = indiana_collegiate_df.append(indiana_collegiate_rows, ignore_index=True)

# add to list of dataframes
df_list.append(indiana_collegiate_df)

# indiana_collegiate_df



C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3459400363.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indiana_collegiate_df = indiana_collegiate_df.append(indiana_collegiate_rows, ignore_index=True)


# Indiana Intercollegiate Athletic Association - precurser to other indiana conference
## ONLY SPONSERED FOOTBALL FROM 1890 TO 1894 SO LIMIT TO THAT TIMEFRAME

The Indiana Intercollegiate Athletic Association was an American college athletic conference established by institutions in the state of Indiana in 1890. At a time when the National Collegiate Athletics Association (NCAA) did not yet exist, such organizations attempted to bring order out of the chaos of the formative years of American intercollegiate sports.

History
The IIAA was founded on March 1, 1890, in a meeting held at the Indianapolis YMCA. The seven charter members were Indiana, Purdue, Butler, Wabash, DePauw, Hanover, and Franklin. Initial officers included W. H. Bliss of Indiana (president), R. D. Meeker of Butler (vice president), and A. H. Bradshaw of Franklin (secretary).[1] The IIAA existed at a time of transition for intercollegiate athletics. When it was created, college sports were only loosely controlled by the institutions they represented; most schools followed the Yale model, in which programs were run by a combination of students, alumni, and boosters. By the time of its dissolution, most larger schools (and an increasing number of smaller ones) had adopted the Chicago model, featuring an athletic director and multi-sport coach who was a full-time employee of the institution, on the model of Amos Alonzo Stagg, whom the University of Chicago had hired for such a role in 1892. Nevertheless, as late as 1903-4, the IIAA was still a student-run organization. That year, C. L. Peck, "an Indianapolis student at Purdue," was elected IIAA president at the association's annual December meeting.[2]

In [19]:
# Indiana Intercollegiate Athletic Association
# oNLY oPERATED AS FOOTBAL CONFERENCE FROM 1890 TO 1894 SO LIMIT SCOPE TO THAT

# need to manually create

# members - Butler, DePauw, Franklin, Hanover, Indiana University, Purdue, Wabash, Rose Polytechnic
# All joined in 1890 and left in 1894
# conference is Indiana IAA

# create rows - leave enrollment and founded blank
butler = {'Institution': 'Butler University', 'Nickname': 'Bulldogs', 'Location': 'Indianapolis, Indiana', 'Founded': '', 'Joined': 1890, 'Left': 1894, 'Enrollment': '', 'Conference': 'Indiana IAA'}
depauw = {'Institution': 'DePauw University', 'Nickname': 'Tigers', 'Location': 'Greencastle, Indiana', 'Founded': '', 'Joined': 1890, 'Left': 1894, 'Enrollment': '', 'Conference': 'Indiana IAA'}
franklin = {'Institution': 'Franklin College', 'Nickname': 'Grizzlies', 'Location': 'Franklin, Indiana', 'Founded': '', 'Joined': 1890, 'Left': 1894, 'Enrollment': '', 'Conference': 'Indiana IAA'}
hanover = {'Institution': 'Hanover College', 'Nickname': 'Panthers', 'Location': 'Hanover, Indiana', 'Founded': '', 'Joined': 1890, 'Left': 1894, 'Enrollment': '', 'Conference': 'Indiana IAA'}
indiana = {'Institution': 'Indiana University', 'Nickname': 'Hoosiers', 'Location': 'Bloomington, Indiana', 'Founded': '', 'Joined': 1890, 'Left': 1894, 'Enrollment': '', 'Conference': 'Indiana IAA'}
purdue = {'Institution': 'Purdue University', 'Nickname': 'Boilermakers', 'Location': 'West Lafayette, Indiana', 'Founded': '', 'Joined': 1890, 'Left': 1894, 'Enrollment': '', 'Conference': 'Indiana IAA'}
wabash = {'Institution': 'Wabash College', 'Nickname': 'Little Giants', 'Location': 'Crawfordsville, Indiana', 'Founded': '', 'Joined': 1890, 'Left': 1894, 'Enrollment': '', 'Conference': 'Indiana IAA'}
rose_polytechnic = {'Institution': 'Rose Polytechnic Institute', 'Nickname': 'Engineers', 'Location': 'Terre Haute, Indiana', 'Founded': '', 'Joined': 1890, 'Left': 1894, 'Enrollment': '', 'Conference': 'Indiana IAA'}

# create blank dataframe
indiana_iaa_df = pd.DataFrame(columns=['Institution', 'Nickname', 'Location', 'Founded', 
           'Joined', 'Left', 'Enrollment', 'Conference'])

# list of rows
indiana_iaa_rows = [butler, depauw, franklin, hanover, indiana, purdue, wabash, rose_polytechnic]

# ADD ROWS TO DATAFRAME
indiana_iaa_df = indiana_iaa_df.append(indiana_iaa_rows, ignore_index=True)

# add to list of dataframes
df_list.append(indiana_iaa_df)

indiana_iaa_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\2478389742.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indiana_iaa_df = indiana_iaa_df.append(indiana_iaa_rows, ignore_index=True)


,Institution,Nickname,Location,Founded,Joined,Left,Enrollment,Conference
0,Butler University,Bulldogs,"Indianapolis, Indiana",,1890,1894,,Indiana IAA
1,DePauw University,Tigers,"Greencastle, Indiana",,1890,1894,,Indiana IAA
2,Franklin College,Grizzlies,"Franklin, Indiana",,1890,1894,,Indiana IAA
3,Hanover College,Panthers,"Hanover, Indiana",,1890,1894,,Indiana IAA
4,Indiana University,Hoosiers,"Bloomington, Indiana",,1890,1894,,Indiana IAA
5,Purdue University,Boilermakers,"West Lafayette, Indiana",,1890,1894,,Indiana IAA
6,Wabash College,Little Giants,"Crawfordsville, Indiana",,1890,1894,,Indiana IAA
7,Rose Polytechnic Institute,Engineers,"Terre Haute, Indiana",,1890,1894,,Indiana IAA


# Interstate Intercollegiate Athletic Conference -
## aka "The Little 19"
### At one time the Illinois Intercollegiate Athletic Conference, or IIAC

The IIAC was formed in April 1908 with eight charter members: Illinois State Normal University (now Illinois State University), Illinois Wesleyan University, Bradley Polytechnic Institute (now Bradley University), Millikin University, Monmouth College, Knox College, Lombard College and Illinois College. The first track meet was held on May 22, 1908. The group quickly expanded. Eastern Illinois State Teachers College (now Eastern Illinois University) and Western Illinois University joined in 1912 and 1914 respectively.

In 1920, the name "Illinois Intercollegiate Athletic Conference" was adopted, providing the initials IIAC. Conference membership reached a peak of 23 member schools in 1928, when virtually all of the small colleges in Illinois were included.

Private schools withdrew during much of the 1930s, until in 1942 only the five state schools remained: Illinois State University, Eastern Illinois University, Northern Illinois University, Southern Illinois University Carbondale and Western Illinois University. In 1950, the league name became the Interstate Intercollegiate Athletic Conference, when Central Michigan University and Eastern Michigan University brought the membership to seven. In 1961-62, Eastern Michigan University and Southern Illinois University Carbondale withdrew; Northern Illinois University followed in 1965-66. The conference disbanded at the end of the 1969–70 academic year.

In [20]:
# Interstate Intercollegiate Athletic Conference -
## aka "The Little 19"
### At one time the Illinois Intercollegiate Athletic Conference, or IIAC

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['IIAC'])

# try to open tables with pandas
iiac_table = pd.read_html(wiki_page.text)

final = iiac_table[2]
former = iiac_table[3]

# append to single dataframe
iiac_df = pd.concat([final, former])
# clean Joined and Left by just takeing first 4 characters
iiac_df['Joined'] = iiac_df['Joined'].str[:4]
iiac_df['Left'] = iiac_df['Left'].str[:4]

# add conference column
iiac_df['Conference'] = 'IIAC'
# keep only the neseccesary columns
iiac_df = iiac_df[keep_cols]

# clean Insitution column of citations
iiac_df['Institution'] = iiac_df['Institution'].str.replace('\[.*\]', '')

# add to list of dataframes
df_list.append(iiac_df)


# final
# former
# iiac_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3247610949.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
  iiac_df['Institution'] = iiac_df['Institution'].str.replace('\[.*\]', '')


# Not Included - should be covered by # Indiana Intercollegiate Athletic Association - table
## Indiana Intercollegiate Conference

The Indiana Intercollegiate Conference (IIC) was an American college athletic conference formed in 1922 to govern intercollegiate competition in male sports in the state of Indiana.

The IIC was the third state-based collegiate athletic conference attempted in Indiana, after the Indiana Intercollegiate Athletic Association (IIAA) and the Indiana College Athletic League (ICAL), but had many more members than its predecessors. Ultimately, the size and diversity of the IIC made it a loosely-run organization and doomed it to fail in the long run. Most of its members ended up joining the Hoosier College Conference (established 1947) or the Indiana Collegiate Conference (established 1950).

# Kansas Collegiate Athletic Conference

On February 15, 1890, the Kansas Intercollegiate Athletic Association was formed; it was the first successful attempt to organize Kansas colleges for the purposes of promoting and regulating amateur intercollegiate athletics. In addition to the private universities and colleges, the conference also included Kansas State Agriculture College (now Kansas State University), the University of Kansas, and Washburn University. In November of that year, the first college football game in Kansas was played between the Kansas Jayhawks and Baker University.[1]

About 1902 the association allied with the Kansas College Athletic Conference, the first group to adopt a definite set of rules and regulations. By the 1920s the conference had changed its name to Kansas Collegiate Athletic Conference and had grown to include 17 regular members and 2 allied members (no longer including the University of Kansas or Kansas State). In 1923 seven colleges withdrew to form the Central Intercollegiate Athletic Conference.

### Only Current Relevant teams are Kansas St and Kansas in very early part of the conference
just manually creating those rows and moving on

In [21]:
# Kansas Collegiate Athletic Conference

# manually create rows for Kansas and Kansas State - 
# K State - Joined in 1902 - Left 1913
# kansas - need to recheck but despite first paragraph of wiki do not see University of Kansas referenced anywhere else

# create row for Kansas State
kansas_state = {'Institution': 'Kansas State University', 'Nickname': 'Wildcats', 'Location': 'Manhattan, Kansas', 'Founded': 1863, 'Joined': 1902, 'Left': 1913, 'Enrollment': 22_221, 'Conference': 'Kansas Collegiate Athletic Conference'}

# create blank dataframe
kansas_collegiate_df = pd.DataFrame(columns=['Institution', 'Nickname', 'Location', 'Founded', 
           'Joined', 'Left', 'Enrollment', 'Conference'])

# add the row to the dataframe
kansas_collegiate_df = kansas_collegiate_df.append(kansas_state, ignore_index=True)

# add to list of dataframes
df_list.append(kansas_collegiate_df)

kansas_collegiate_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\311962313.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kansas_collegiate_df = kansas_collegiate_df.append(kansas_state, ignore_index=True)


,Institution,Nickname,Location,Founded,Joined,Left,Enrollment,Conference
0,Kansas State University,Wildcats,"Manhattan, Kansas",1863,1902,1913,22221,Kansas Collegiate Athletic Conference


# The River States Conference (RSC)

The River States Conference (RSC), formerly known as the Kentucky Intercollegiate Athletic Conference (KIAC), is a college athletic conference affiliated with the National Association of Intercollegiate Athletics (NAIA). Although it was historically a Kentucky-only conference, it has now expanded to include members in Indiana, Ohio, Pennsylvania, and West Virginia, and at various times in the past has also had members in Missouri, Tennessee, and Virginia.

In March 2016, the KIAC announced it would change its name to the River States Conference, effective July 1, 2016, to better reflect its membership, which has expanded beyond Kentucky and now includes members in Pennsylvania, Indiana, Ohio, and West Virginia.[1]

In [22]:
# Kentucky Intercollegiate Athletic Conference (KIAC) - ignoring the modern version because no relevant teams
# just going to grab former members from the wiki page

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['KIAC'])

# try to open tables with pandas
kiac_table = pd.read_html(wiki_page.text)

kiac_df = kiac_table[2]

# Rename coumns (Left[b] to Left and Joined[a] to Joined)
kiac_df = kiac_df.rename(columns={'Left[b]': 'Left', 'Joined[a]': 'Joined'})

# Get citations out of Institution column
kiac_df['Institution'] = kiac_df['Institution'].str.replace('\[.*\]', '')

# get rid of citation in Joined and Left columns
kiac_df['Joined'] = kiac_df['Joined'].str.replace('\[.*\]', '')
kiac_df['Left'] = kiac_df['Left'].str.replace('\[.*\]', '')

# add conference column
kiac_df['Conference'] = 'Kentucky IAC'

# keep only the neseccesary columns
kiac_df = kiac_df[keep_cols]

# add to list of dataframes
df_list.append(kiac_df)




# kiac_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3466421347.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  kiac_df['Institution'] = kiac_df['Institution'].str.replace('\[.*\]', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3466421347.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  kiac_df['Joined'] = kiac_df['Joined'].str.replace('\[.*\]', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3466421347.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  kiac_df['Left'] = kiac_df['Left'].str.replace('\[.*\]', '')


# Lone Star Conference - currently just D2 teams but relevant past

The Lone Star Conference (LSC) is a college athletic conference affiliated with the National Collegiate Athletic Association (NCAA) at the Division II level. Member institutions are located in the southwestern United States, with schools in Texas, Oklahoma, New Mexico, and Arkansas, with two members in the Pacific Northwest states of Oregon and Washington competing as affiliates for football only.

The Lone Star Conference operates from the same headquarters complex in the Dallas suburb of Richardson as the American Southwest Conference.

History
The conference was formed in 1931 when five schools withdrew from the old Texas Intercollegiate Athletic Association. Charter members included East Texas State (now Texas A&M–Commerce), North Texas State (now University of North Texas), Sam Houston State, Southwest Texas State (now Texas State), and Stephen F. Austin. With Texas A&M–Commerce starting its transition to Division I in July 2022, none of the five charter members remain in Division II or in the conference – all have moved up to Division I (in football as of 2023, North Texas, Sam Houston, and Texas State compete in NCAA Division I FBS, while Stephen F. Austin and Texas A&M–Commerce compete in NCAA Division I FCS).

In [23]:
# Lone Star Conference

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['Lone Star'])

# try to open tables with pandas
lone_star_table = pd.read_html(wiki_page.text)

current = lone_star_table[1]
former = lone_star_table[4]

# append to single dataframe
lone_star_df = pd.concat([current, former])

# add conference column
lone_star_df['Conference'] = 'Lone Star'
current
former

lone_star_df

# need to Correc the following Institution rows: Cameron University, Midwestern State University, West Texas A&M University, 
# Tarleton State University and Trinity University - each has multiple values in Joind and/or Left columns
# Ranges are as follows:
# Cameron University - (1988 - 1993 and 1996 -present)
# Midwestern State University - (Joined for just one year 1948, then joined in 1995- present)
# West Texas A&M University - (Joined in 1986 - 1990 and 1996 - present)
# Tarleton State University - (1968-1976, 1994-2020)
# Trinity University - (1932-1934 and 1946-1949)

# need to drop the original rows and add the new rows
# Cameron University
lone_star_df = lone_star_df[lone_star_df['Institution'] != 'Cameron University']
# create new rows
cameron1 = {'Institution': 'Cameron University', 'Nickname': 'Aggies', 'Location': 'Lawton, Oklahoma', 'Founded': 1908, 'Joined': 1988, 'Left': 1993, 'Enrollment': 5_500, 'Conference': 'Lone Star'}
cameron2 = {'Institution': 'Cameron University', 'Nickname': 'Aggies', 'Location': 'Lawton, Oklahoma', 'Founded': 1908, 'Joined': 1996, 'Left': '', 'Enrollment': 5_500, 'Conference': 'Lone Star'}
# Midwestern State University
lone_star_df = lone_star_df[lone_star_df['Institution'] != 'Midwestern State University']
# create new rows
midwestern1 = {'Institution': 'Midwestern State University', 'Nickname': 'Mustangs', 'Location': 'Wichita Falls, Texas', 'Founded': 1922, 'Joined': 1948, 'Left': '', 'Enrollment': 6_043, 'Conference': 'Lone Star'}
midwestern2 = {'Institution': 'Midwestern State University', 'Nickname': 'Mustangs', 'Location': 'Wichita Falls, Texas', 'Founded': 1922, 'Joined': 1995, 'Left': '', 'Enrollment': 6_043, 'Conference': 'Lone Star'}

# West Texas A&M University
lone_star_df = lone_star_df[lone_star_df['Institution'] != 'West Texas A&M University']
# create new rows
west_texas1 = {'Institution': 'West Texas A&M University', 'Nickname': 'Buffaloes', 'Location': 'Canyon, Texas', 'Founded': 1910, 'Joined': 1986, 'Left': 1990, 'Enrollment': 10_029, 'Conference': 'Lone Star'}
west_texas2 = {'Institution': 'West Texas A&M University', 'Nickname': 'Buffaloes', 'Location': 'Canyon, Texas', 'Founded': 1910, 'Joined': 1996, 'Left': '', 'Enrollment': 10_029, 'Conference': 'Lone Star'}

# Tarleton State University
lone_star_df = lone_star_df[lone_star_df['Institution'] != 'Tarleton State University']
# create new rows
tarleton1 = {'Institution': 'Tarleton State University', 'Nickname': 'Texans', 'Location': 'Stephenville, Texas', 'Founded': 1899, 'Joined': 1968, 'Left': 1976, 'Enrollment': 13_109, 'Conference': 'Lone Star'}
tarleton2 = {'Institution': 'Tarleton State University', 'Nickname': 'Texans', 'Location': 'Stephenville, Texas', 'Founded': 1899, 'Joined': 1994, 'Left': 2020, 'Enrollment': 13_109, 'Conference': 'Lone Star'}

# Trinity University
lone_star_df = lone_star_df[lone_star_df['Institution'] != 'Trinity University']
# create new rows
trinity1 = {'Institution': 'Trinity University', 'Nickname': 'Tigers', 'Location': 'San Antonio, Texas', 'Founded': 1869, 'Joined': 1932, 'Left': 1934, 'Enrollment': 2_643, 'Conference': 'Lone Star'}
trinity2 = {'Institution': 'Trinity University', 'Nickname': 'Tigers', 'Location': 'San Antonio, Texas', 'Founded': 1869, 'Joined': 1946, 'Left': 1949, 'Enrollment': 2_643, 'Conference': 'Lone Star'}

# create list of rows
lone_star_rows = [cameron1, cameron2, midwestern1, midwestern2, west_texas1, west_texas2, tarleton1, tarleton2, trinity1, trinity2]

# append rows to df
lone_star_df = lone_star_df.append(lone_star_rows, ignore_index=True)

# drop unnecessary columns
lone_star_df = lone_star_df[keep_cols]

# remove citations from Institution column
lone_star_df['Institution'] = lone_star_df['Institution'].str.replace('\[.*\]', '')






# add to list of dataframes
df_list.append(lone_star_df)


# lone_star_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\1082514418.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lone_star_df = lone_star_df.append(lone_star_rows, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\1082514418.py:71: FutureWarning: The default value of regex will change from True to False in a future version.
  lone_star_df['Institution'] = lone_star_df['Institution'].str.replace('\[.*\]', '')


# Summit League

The Summit League, or The Summit, is an NCAA Division I intercollegiate athletic conference with its membership mostly located in the Midwestern United States, from Minnesota in the east, to the Dakotas and Nebraska to the West, and Missouri in the South, with additional members in the Western state of Colorado and the Southern state of Oklahoma. Founded as the Association of Mid-Continent Universities in 1982,[1] it rebranded as the Mid-Continent Conference in 1989,[3] then again as the Summit League on June 1, 2007.[4] The league headquarters are in Sioux Falls, South Dakota.

The membership currently consists of 9 full members plus 7 associate members. The most recent change in the core conference membership is the 2021 arrival of the University of St. Thomas, which began an unprecedented transition from NCAA Division III to Division I.[5] A year earlier, the University of Missouri–Kansas City returned as a full member after a seven-year absence with the new athletic identity of the Kansas City Roos,[6] while Purdue University Fort Wayne left for the Horizon League.[7] A total of 32 schools have been full members; the last charter member remaining in the league, Western Illinois University, left for the Ohio Valley Conference on July 1, 2023, though it will remain in the Summit men's soccer league through the fall 2023 season.[8]

## History

### Early Days

The conference can trace its roots back to 1978, when the Mid-Continent Athletic Association was founded as a football-only conference playing in Division II at the time.[9] Its inaugural members were the University of Akron, Eastern Illinois University, University of Northern Iowa, Northern Michigan University, Western Illinois University, and Youngstown State University; Wayne State University had also expressed interest in joining, but ultimately never did.[10] Akron left after the 1979 season, while Northern Michigan and Youngstown State left the following year; they were replaced by Southwest Missouri State (now known as Missouri State University) in 1981. The 1981 season also saw the conference as a whole move from Division II to Division I-AA; this would be the conference's final season under the name of the Mid-Continent Athletic Association.

In [24]:
# Mid-Continent Conference - now called the SUmmit League

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['MCC'])

# try to open tables with pandas
mcc_table = pd.read_html(wiki_page.text)

current = mcc_table[1]
former = mcc_table[4]

# append to single dataframe
mcc_df = pd.concat([current, former])

# add conference column
mcc_df['Conference'] = 'Mid-Continent (MCC)'

# Correct Two schools that have multiple values in Joined and Left columns
# University of Missouri–Kansas City (1994-2012 and 2020-present)
# Oral Roberts University (1997-2011 and 2014-present)

# drop the original rows and add the new rows
# University of Missouri–Kansas City
mcc_df = mcc_df[mcc_df['Institution'] != 'University of Missouri–Kansas City']
# create new rows
umkc1 = {'Institution': 'University of Missouri–Kansas City', 'Nickname': 'Roos', 'Location': 'Kansas City, Missouri', 'Founded': 1933, 'Joined': 1994, 'Left': 2012, 'Enrollment': 16_000, 'Conference': 'Mid-Continent (MCC)'}
umkc2 = {'Institution': 'University of Missouri–Kansas City', 'Nickname': 'Roos', 'Location': 'Kansas City, Missouri', 'Founded': 1933, 'Joined': 2020, 'Left': '', 'Enrollment': 16_000, 'Conference': 'Mid-Continent (MCC)'}

# Oral Roberts University
mcc_df = mcc_df[mcc_df['Institution'] != 'Oral Roberts University']
# create new rows
oral_roberts1 = {'Institution': 'Oral Roberts University', 'Nickname': 'Golden Eagles', 'Location': 'Tulsa, Oklahoma', 'Founded': 1963, 'Joined': 1997, 'Left': 2011, 'Enrollment': 4_000, 'Conference': 'Mid-Continent (MCC)'}
oral_roberts2 = {'Institution': 'Oral Roberts University', 'Nickname': 'Golden Eagles', 'Location': 'Tulsa, Oklahoma', 'Founded': 1963, 'Joined': 2014, 'Left': '', 'Enrollment': 4_000, 'Conference': 'Mid-Continent (MCC)'}

# create list of rows
mcc_rows = [umkc1, umkc2, oral_roberts1, oral_roberts2]

# append rows to df
mcc_df = mcc_df.append(mcc_rows, ignore_index=True)

# Correct University of St. Thomas - Joined in 2021
# replace value in existing row
mcc_df.loc[mcc_df['Institution'] == 'University of St. Thomas', 'Joined'] = 2021


# keep only the neseccesary columns
mcc_df = mcc_df[keep_cols]

# add to list of dataframes
df_list.append(mcc_df)

# current
# # former
# mcc_df


C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\1134067802.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mcc_df = mcc_df.append(mcc_rows, ignore_index=True)


# Michigan Intercollegiate Athletic Association
## Oldest Conference In America - now competes exclusively in D3
Still want to scrape for relevent teams

## History
The Michigan Intercollegiate Athletic Association (MIAA) is an athletic conference that competes in the NCAA's Division III. There are nine teams in the conference, all located in the states of Michigan and Indiana. The Michigan Intercollegiate Athletic Association was established on March 24, 1888, making it the oldest college athletic conference in the United States.[1]

The MIAA conference was established on March 24, 1888, and is the oldest collegiate athletic conference in the United States. The number of sports with competition is 23 (12 men and 11 women sports). T


In [25]:
# Michigan Intercollegiate Athletic Association
## contains Michigan State and Eastern Michigan under old names

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['MIAA'])

# try to open tables with pandas
miaa_table = pd.read_html(wiki_page.text)

current = miaa_table[4]
former = miaa_table[5]

# append to single dataframe
miaa_df = pd.concat([current, former])

# correct the State Agricultural College of Michigan[c] to Michigan State University & Michigan State Normal College[a] to Eastern Michigan
miaa_df['Institution'] = miaa_df['Institution'].str.replace('State Agricultural College of Michigan\[c\]', 'Michigan State University')
miaa_df['Institution'] = miaa_df['Institution'].str.replace('Michigan State Normal College\[a\]', 'Eastern Michigan University')




# Defiance College & hillsdale College - keep only the first 4 character of Joined and Left
miaa_df['Joined'] = miaa_df['Joined'].str[:4]
miaa_df['Left'] = miaa_df['Left'].str[:4]

# add conference column
miaa_df['Conference'] = 'Michigan IAA'

# keep only the neseccesary columns
miaa_df = miaa_df[keep_cols]

# Eastern Michgian - 2 rows (1892-1901 and 1920-1925)
# delete orig row
miaa_df = miaa_df[miaa_df['Institution'] != 'Eastern Michigan University']
# create new rows
eastern1 = {'Institution': 'Eastern Michigan University', 'Nickname': 'Eagles', 'Location': 'Ypsilanti, Michigan', 'Founded': 1849, 'Joined': 1892, 'Left': 1901, 'Enrollment': 20_000, 'Conference': 'Michigan IAA'}
eastern2 = {'Institution': 'Eastern Michigan University', 'Nickname': 'Eagles', 'Location': 'Ypsilanti, Michigan', 'Founded': 1849, 'Joined': 1920, 'Left': 1925, 'Enrollment': 20_000, 'Conference': 'Michigan IAA'}



# drop orig Adrian College row
miaa_df = miaa_df[miaa_df['Institution'] != 'Adrian College']
# Create new rows for Adrian College (1908-1922 and 1937 - present)
adrian1 = {'Institution': 'Adrian College', 'Nickname': 'Bulldogs', 'Location': 'Adrian, Michigan', 'Founded': 1859, 'Joined': 1908, 'Left': 1922, 'Enrollment': 1_700, 'Conference': 'Michigan IAA'}
adrian2 = {'Institution': 'Adrian College', 'Nickname': 'Bulldogs', 'Location': 'Adrian, Michigan', 'Founded': 1859, 'Joined': 1937, 'Left': '', 'Enrollment': 1_700, 'Conference': 'Michigan IAA'}


# University of Olivet[b] (1888-1939 and 1952-present)
# drop orig Olivet College row
miaa_df = miaa_df[miaa_df['Institution'] != 'The University of Olivet[b]']
# Create new rows for Olivet College (1888-1939 and 1952-present)
olivet1 = {'Institution': 'Olivet College', 'Nickname': 'Comets', 'Location': 'Olivet, Michigan', 'Founded': 1844, 'Joined': 1888, 'Left': 1939, 'Enrollment': 1_100, 'Conference': 'Michigan IAA'}
olivet2 = {'Institution': 'Olivet College', 'Nickname': 'Comets', 'Location': 'Olivet, Michigan', 'Founded': 1844, 'Joined': 1952, 'Left': '', 'Enrollment': 1_100, 'Conference': 'Michigan IAA'}

# correct the Joined and Left columns
# Correct Bad Values in Single COlumns
# Michigan state - Joined in 1998 - Left 1906
miaa_df.loc[miaa_df['Institution'] == 'Michigan State University', 'Joined'] = 1898
miaa_df.loc[miaa_df['Institution'] == 'Michigan State University', 'Left'] = 1906
# add new rows to list
miaa_rows = [adrian1, adrian2, olivet1, olivet2, eastern1, eastern2]
# add rows to df
miaa_df = miaa_df.append(miaa_rows, ignore_index=True)


# add to list of dataframes
df_list.append(miaa_df)

# current
# former
# miaa_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\653137582.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  miaa_df['Institution'] = miaa_df['Institution'].str.replace('State Agricultural College of Michigan\[c\]', 'Michigan State University')
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\653137582.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  miaa_df['Institution'] = miaa_df['Institution'].str.replace('Michigan State Normal College\[a\]', 'Eastern Michigan University')
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\653137582.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  miaa_df = miaa_df.append(miaa_rows, ignore_index=True)


# Maryland Intercollegiate Football Association

The Maryland Intercollegiate Football Association (MIFA), also called the Maryland Intercollegiate League, was an early college football conference with a membership composed of schools located primarily in the state of Maryland. One exception was Gallaudet, which is located in Washington, D.C. All of the members were universities, except Baltimore City College, a preparatory school located in Baltimore, Maryland.

The impetus for the formation of the MIFA was a controversial game on November 1, 1893 between the Maryland Agricultural College (now known as the University of Maryland, College Park) and Saint John's College of Annapolis. Maryland won the game, 6–0, but St. John's players later wrote in the Baltimore American that "a decision by which the M.A.C. were allowed to score the only touchdown made by the quarterback after a run of 90 yards, with no one in pursuit, appeared a very doubtful one."[1] Maryland finished the season by beating Western Maryland and the Orient Athletic Club. With a 6–0 record, the "Aggies" were named the state champions, but the title was contested by St. John's. The following season, Maryland's former quarterback and coach William W. Skinner led the effort to create an intercollegiate football league to improve the process in which the state championship was awarded.[2] The conference began competition on October 13, 1894 with games between St. John's College and Washington College in Annapolis, and Maryland Agricultural College and Western Maryland in Westminster.[3]

The association voted to disband during the 1899 season under the threat of withdrawal by Johns Hopkins University after legislation was passed to prevent its use of graduate student-athletes. The league was subsequently reorganized without Hopkins. Maryland Agricultural College claimed the championship in 1906. After the 1907 season, the Aggies, St. John's and Washington reformed the association, without Western Maryland College, with the S.J.C. Cadets retaining their title of the three-team "league" in 1908.


In [26]:
# Maryland Intercollegiate Football Association
# Need to create data manually - no tables on wiki page

# Membership
# The following schools were members of the MIFA at some point. Where known, the years of membership are included:[2][4][5]

# Baltimore City College, 189?-1897[6]
# Gallaudet, 1897–?[6]
# Johns Hopkins, 1894–1899[7]
# Loyola
# Maryland Agricultural College (now University of Maryland, College Park), 1894–1897[8][disputed – discuss]
# Mount St. Mary's
# St. John's College, 1894–?[3]
# Washington College, 1894–1897[3][6]
# Western Maryland (now McDaniel College), 1894–?[3]

# create rows
baltimore = {'Institution': 'Baltimore City College', 'Nickname': 'Black Knights', 'Location': 'Baltimore, Maryland', 'Founded': 1839, 'Joined': 1894, 'Left': 1897, 'Enrollment': 1_500, 'Conference': 'Maryland Intercollegiate Football Association'}
# gallaudet = {'Institution': 'Gallaudet University', 'Nickname': 'Bison', 'Location': 'Washington, D.C.', 'Founded': 1864, 'Joined': 1897, 'Left': '', 'Enrollment': 1_100, 'Conference': 'Maryland Intercollegiate Football Association'}
johns_hopkins = {'Institution': 'Johns Hopkins University', 'Nickname': 'Blue Jays', 'Location': 'Baltimore, Maryland', 'Founded': 1876, 'Joined': 1894, 'Left': 1899, 'Enrollment': 24_000, 'Conference': 'Maryland Intercollegiate Football Association'}
# loyola = {'Institution': 'Loyola University Maryland', 'Nickname': 'Greyhounds', 'Location': 'Baltimore, Maryland', 'Founded': 1852, 'Joined': '', 'Left': '', 'Enrollment': 6_000, 'Conference': 'Maryland Intercollegiate Football Association'}
maryland_agricultural = {'Institution': 'University of Maryland, College Park', 'Nickname': 'Terrapins', 'Location': 'College Park, Maryland', 'Founded': 1856, 'Joined': 1894, 'Left': 1897, 'Enrollment': 41_200, 'Conference': 'Maryland Intercollegiate Football Association'}
# mount_st_marys = {'Institution': 'Mount St. Marys University', 'Nickname': 'Mountaineers', 'Location': 'Emmitsburg, Maryland', 'Founded': 1808, 'Joined': '', 'Left': '', 'Enrollment': 2_300, 'Conference': 'Maryland Intercollegiate Football Association'}
# st_johns = {'Institution': 'St. John\'s College', 'Nickname': 'Johnnies', 'Location': 'Annapolis, Maryland', 'Founded': 1696, 'Joined': 1894, 'Left': '', 'Enrollment': 500, 'Conference': 'Maryland Intercollegiate Football Association'}
washington_college = {'Institution': 'Washington College', 'Nickname': 'Shoremen', 'Location': 'Chestertown, Maryland', 'Founded': 1782, 'Joined': 1894, 'Left': 1897, 'Enrollment': 1_450, 'Conference': 'Maryland Intercollegiate Football Association'}
# western_maryland = {'Institution': 'McDaniel College', 'Nickname': 'Green Terror', 'Location': 'Westminster, Maryland', 'Founded': 1867, 'Joined': 1894, 'Left': '', 'Enrollment': 3_000, 'Conference': 'Maryland Intercollegiate Football Association'}

## COMMENTED OUT SCHOOLS WITHOUT GOOD DATA _ ALL ENROOLMENT DATA IS ASSUMED
# create blank dataframe
mifa_df = pd.DataFrame(columns=['Institution', 'Nickname', 'Location', 'Founded', 
           'Joined', 'Left', 'Enrollment', 'Conference'])

# list of rows
mifa_rows = [baltimore, johns_hopkins, maryland_agricultural, washington_college]

# ADD ROWS TO DATAFRAME
mifa_df = mifa_df.append(mifa_rows, ignore_index=True)

# Change conference name
mifa_df['Conference'] = 'Maryland IFA'
# clear enrollment column
mifa_df['Enrollment'] = ''

# add to list of dataframes
df_list.append(mifa_df)

# mifa_df


C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\4001143281.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mifa_df = mifa_df.append(mifa_rows, ignore_index=True)


# Middle Atlantic Conference

### Includes a number of FBS schools in the 1950s

The following is an incomplete list of former members of the U.S. college athletic league now known as the Middle Atlantic Conferences (MAC). This includes schools that were members under the MAC's previous identity as the Middle Atlantic States Collegiate Athletics Conference.

One school which had departed the conference has since re-joined: Stevens Institute of Technology, which competed in the MAC from 1922–23 to 1977–78, returned back since the 2019–20 school year, joining the MAC's Freedom Conference.[1]

In [27]:
# Middle Atlantic Conference

url = 'https://en.wikipedia.org/wiki/List_of_former_Middle_Atlantic_Conference_members'

# get the tables from the wiki page
tables = pd.read_html(url)

# remove all citations from the Institution column
tables[0]['Institution'] = tables[0]['Institution'].str.replace('\[.*\]', '')
# clean Joined and Left by only keeping first 4 characters
tables[0]['Joined'] = tables[0]['Joined'].str[:4]
tables[0]['Left'] = tables[0]['Left'].str[:4]

# add conference column
tables[0]['Conference'] = 'Middle Atlantic'

# keep only the neseccesary columns
mid_atlantic_df = tables[0][keep_cols]

# add to list of dataframes
df_list.append(mid_atlantic_df)


# mid_atlantic_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\4289051821.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  tables[0]['Institution'] = tables[0]['Institution'].str.replace('\[.*\]', '')


## Far West Conference - currently the Northern California Athletic Conference

The NCAC was founded as the Far Western Conference (FWC) in 1925 by its charter member schools: Fresno State, Saint Mary's, UC Davis, Nevada, San Jose State and College of the Pacific.[2]

Nevada's departure from the conference in 1940 left the conference with only four members: Chico State, Fresno State, College of the Pacific and UC Davis.[3] The conference looked to four nominees in Humboldt State (joined in 1940), San Francisco State, Santa Barbara State College (later UC Santa Barbara) and California Poly of San Luis Obispo.[4]

Shortly after World War II, the remaining members, with the exception of UC Davis, Chico State and Humboldt State, would leave for other conferences, to be replaced over the years by San Francisco State (joined in 1946), Southern Oregon (1947), Sacramento State (1953), Hayward State (1961) and Sonoma State (1966). During the 1990s, each of the universities associated with the California State system chose to disband their football teams in order to comply with Title IX, with the exception of Humboldt State, which added two women's sports to achieve compliance, and Sacramento State.[1]

In [28]:
# Far West Conference

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['NCAC'])

# try to open tables with pandas
ncac_table = pd.read_html(wiki_page.text)

far_west_df = ncac_table[1]

# Correct Nevada row - multiple entries (1925-1940 and 1953-1969)
# San Francisco State - multiple entries (1946-1951 and 1953-1998)
# Sonoma State - multiple entries (1966-1974 and 1982-1998)

# Nevada - delete orig row
far_west_df = far_west_df[far_west_df['Institution'] != 'University of Nevada']
# replace with 2 new rows
nevada1 = {'Institution': 'University of Nevada', 'Nickname': 'Wolf Pack', 'Location': 'Reno, Nevada', 'Founded': 1874, 'Joined': 1925, 'Left': 1940, 'Enrollment': 21_463, 'Conference': 'Far Western'}
nevada2 = {'Institution': 'University of Nevada', 'Nickname': 'Wolf Pack', 'Location': 'Reno, Nevada', 'Founded': 1874, 'Joined': 1953, 'Left': 1969, 'Enrollment': 21_463, 'Conference': 'Far Western'}

# San Francisco State - delete orig row
far_west_df = far_west_df[far_west_df['Institution'] != 'San Francisco State University']
# replace with 2 new rows
san_francisco1 = {'Institution': 'San Francisco State University', 'Nickname': 'Gators', 'Location': 'San Francisco, California', 'Founded': 1899, 'Joined': 1946, 'Left': 1951, 'Enrollment': 29_778, 'Conference': 'Far Western'}
san_francisco2 = {'Institution': 'San Francisco State University', 'Nickname': 'Gators', 'Location': 'San Francisco, California', 'Founded': 1899, 'Joined': 1953, 'Left': 1998, 'Enrollment': 29_778, 'Conference': 'Far Western'}

# Sonoma State - delete orig row
far_west_df = far_west_df[far_west_df['Institution'] != 'Sonoma State University']
# replace with 2 new rows
sonoma1 = {'Institution': 'Sonoma State University', 'Nickname': 'Seawolves', 'Location': 'Rohnert Park, California', 'Founded': 1960, 'Joined': 1966, 'Left': 1974, 'Enrollment': 9_300, 'Conference': 'Far Western'}
sonoma2 = {'Institution': 'Sonoma State University', 'Nickname': 'Seawolves', 'Location': 'Rohnert Park, California', 'Founded': 1960, 'Joined': 1982, 'Left': 1998, 'Enrollment': 9_300, 'Conference': 'Far Western'}

# create list of rows
far_west_rows = [nevada1, nevada2, san_francisco1, san_francisco2, sonoma1, sonoma2]

# append rows to df
far_west_df = far_west_df.append(far_west_rows, ignore_index=True)

# create blank enroolment column
far_west_df['Enrollment'] = ''

# add conference column
far_west_df['Conference'] = 'Far Western'

# keep only the neseccesary columns
far_west_df = far_west_df[keep_cols]

# drop Notre Dame (CA)	 to avoid confusion with Notre Dame
far_west_df = far_west_df[far_west_df['Institution'] != 'Notre Dame (CA)']

# add to list of dataframes
df_list.append(far_west_df)


# far_west_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\1641292477.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  far_west_df = far_west_df.append(far_west_rows, ignore_index=True)


# Ohio Athletic Conference
The Ohio Athletic Conference was found in 1902 with six charter members—Case Tech, Kenyon, Oberlin, Ohio State, Ohio Wesleyan, and Western Reserve. By 1934, the conference reached an all-time high of twenty-four members,[1] seeing many schools come and go throughout the upcoming decades. By 2000, the conference solidified to its current form with the addition of its final school, Wilmington, to ten members.

In [29]:
# Ohio Athletic Conference

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['OAC'])

# try to open tables with pandas
oac_table = pd.read_html(wiki_page.text)

current = oac_table[1]
former = oac_table[2]

# append to single dataframe
oac_df = pd.concat([current, former])

# remove citations from Institution column
oac_df['Institution'] = oac_df['Institution'].str.replace('\[.*\]', '')
# remove citations from Founded column
oac_df['Founded'] = oac_df['Founded'].str.replace('\[.*\]', '')

# add conference column
oac_df['Conference'] = 'Ohio Athletic'

# keep only the neseccesary columns
oac_df = oac_df[keep_cols]

## Corrections to Make
# Baldwin Wallace - (1915-1918, 1923-1947, 1961-present)
# John Carroll - (1932-1948, 1989-present)
# Ohio Northern - (1916-1948, 1973-present)
# University of Akron - (1915-1936, 1944-1966)
# Denison University - (1907-1928, 1933-1984)
# Hiram College - (1920-1935, 1951-1971, 1989-1999)
# Ohio Wesleyan - (1902-1928, 1947-1984)
# Wittenberg University - (1909-1928, 1934-1989)

# Delete all the old rows
oac_df = oac_df[oac_df['Institution'] != 'Baldwin Wallace University']
oac_df = oac_df[oac_df['Institution'] != 'John Carroll University']
oac_df = oac_df[oac_df['Institution'] != 'Ohio Northern University']
oac_df = oac_df[oac_df['Institution'] != 'University of Akron']
oac_df = oac_df[oac_df['Institution'] != 'Denison University']
oac_df = oac_df[oac_df['Institution'] != 'Hiram College']
oac_df = oac_df[oac_df['Institution'] != 'Ohio Wesleyan University']
oac_df = oac_df[oac_df['Institution'] != 'Wittenberg University']

# create new rows
baldwin1 = {'Institution': 'Baldwin Wallace University', 'Nickname': 'Yellow Jackets', 'Location': 'Berea, Ohio', 'Founded': 1845, 'Joined': 1915, 'Left': 1918, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
baldwin2 = {'Institution': 'Baldwin Wallace University', 'Nickname': 'Yellow Jackets', 'Location': 'Berea, Ohio', 'Founded': 1845, 'Joined': 1923, 'Left': 1947, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
baldwin3 = {'Institution': 'Baldwin Wallace University', 'Nickname': 'Yellow Jackets', 'Location': 'Berea, Ohio', 'Founded': 1845, 'Joined': 1961, 'Left': '', 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
john_carroll1 = {'Institution': 'John Carroll University', 'Nickname': 'Blue Streaks', 'Location': 'University Heights, Ohio', 'Founded': 1886, 'Joined': 1932, 'Left': 1948, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
john_carroll2 = {'Institution': 'John Carroll University', 'Nickname': 'Blue Streaks', 'Location': 'University Heights, Ohio', 'Founded': 1886, 'Joined': 1989, 'Left': '', 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
ohio_northern1 = {'Institution': 'Ohio Northern University', 'Nickname': 'Polar Bears', 'Location': 'Ada, Ohio', 'Founded': 1871, 'Joined': 1916, 'Left': 1948, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
ohio_northern2 = {'Institution': 'Ohio Northern University', 'Nickname': 'Polar Bears', 'Location': 'Ada, Ohio', 'Founded': 1871, 'Joined': 1973, 'Left': '', 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
akron1 = {'Institution': 'University of Akron', 'Nickname': 'Zips', 'Location': 'Akron, Ohio', 'Founded': 1870, 'Joined': 1915, 'Left': 1936, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
akron2 = {'Institution': 'University of Akron', 'Nickname': 'Zips', 'Location': 'Akron, Ohio', 'Founded': 1870, 'Joined': 1944, 'Left': 1966, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
denison1 = {'Institution': 'Denison University', 'Nickname': 'Big Red', 'Location': 'Granville, Ohio', 'Founded': 1831, 'Joined': 1907, 'Left': 1928, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
denison2 = {'Institution': 'Denison University', 'Nickname': 'Big Red', 'Location': 'Granville, Ohio', 'Founded': 1831, 'Joined': 1933, 'Left': 1984, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
hiram1 = {'Institution': 'Hiram College', 'Nickname': 'Terriers', 'Location': 'Hiram, Ohio', 'Founded': 1850, 'Joined': 1920, 'Left': 1935, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
hiram2 = {'Institution': 'Hiram College', 'Nickname': 'Terriers', 'Location': 'Hiram, Ohio', 'Founded': 1850, 'Joined': 1951, 'Left': 1971, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
hiram3 = {'Institution': 'Hiram College', 'Nickname': 'Terriers', 'Location': 'Hiram, Ohio', 'Founded': 1850, 'Joined': 1989, 'Left': 1999, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
ohio_wesleyan1 = {'Institution': 'Ohio Wesleyan University', 'Nickname': 'Battling Bishops', 'Location': 'Delaware, Ohio', 'Founded': 1842, 'Joined': 1902, 'Left': 1928, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
ohio_wesleyan2 = {'Institution': 'Ohio Wesleyan University', 'Nickname': 'Battling Bishops', 'Location': 'Delaware, Ohio', 'Founded': 1842, 'Joined': 1947, 'Left': 1984, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
wittenberg1 = {'Institution': 'Wittenberg University', 'Nickname': 'Tigers', 'Location': 'Springfield, Ohio', 'Founded': 1845, 'Joined': 1909, 'Left': 1928, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}
wittenberg2 = {'Institution': 'Wittenberg University', 'Nickname': 'Tigers', 'Location': 'Springfield, Ohio', 'Founded': 1845, 'Joined': 1934, 'Left': 1989, 'Enrollment': 3_500, 'Conference': 'Ohio Athletic'}

# create list of new rows
oac_rows = [baldwin1, baldwin2, baldwin3, john_carroll1, john_carroll2, ohio_northern1, ohio_northern2, akron1, akron2, denison1, denison2, hiram1, hiram2, hiram3, ohio_wesleyan1, ohio_wesleyan2, wittenberg1, wittenberg2]

# append rows to df
oac_df = oac_df.append(oac_rows, ignore_index=True)

# Change Miami University to Miami (OH)
oac_df.loc[oac_df['Institution'] == 'Miami University', 'Institution'] = 'Miami (OH)'

# add to list of dataframes
df_list.append(oac_df)

# current
# # former
# oac_df


C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3218646512.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  oac_df['Institution'] = oac_df['Institution'].str.replace('\[.*\]', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3218646512.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  oac_df['Founded'] = oac_df['Founded'].str.replace('\[.*\]', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3218646512.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  oac_df = oac_df.append(oac_rows, ignore_index=True)


# Ohio Valley Conference

The Ohio Valley Conference can trace its roots to 1941 when Murray State athletic director Roy Stewart, Eastern Kentucky athletic director Charles "Turkey" Hughes, and Western Kentucky public relations director Kelly Thompson first formulated the idea of establishing a regional athletics conference. The plan was put on hold due to World War II, but it was resurrected after the conclusion of the war. In 1948, the three schools joined with Louisville, Morehead State, and Evansville to form the Ohio Valley Conference. While many collegiate conferences are struggling today with the question of whether their policies and rules should be determined by the athletic departments or by the institutional heads, from the very beginning, the OVC has been run by the presidents of its member schools.

In [30]:
# Ohio Valley Conference

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['OVC'])

# try to open tables with pandas
ovc_table = pd.read_html(wiki_page.text)

current = ovc_table[1]
former = ovc_table[3]
football = ovc_table[4]
# keep only Murray State from football table
football = football[football['Institution'] == 'Murray State University']

# append to a single dataframe
ovc_df = pd.concat([current, former, football])

# add conference column
ovc_df['Conference'] = 'Ohio Valley (OVC)'

# keep only the neseccesary columns
ovc_df = ovc_df[keep_cols]

# drop any citation from Institution column
ovc_df['Institution'] = ovc_df['Institution'].str.replace('\[.*\]', '')
# drop any citations from Joined & Enrolled columns
# ovc_df['Joined'] = ovc_df['Joined'].str.replace('\[.*\]', '')
# ovc_df['Enrollment'] = ovc_df['Enrollment'].str.replace('\[.*\]', '')

# correct Murrey State - Joined in 1948 left in 2022
ovc_df.loc[ovc_df['Institution'] == 'Murray State University', 'Left'] = 2022
ovc_df.loc[ovc_df['Institution'] == 'Murray State University', 'Joined'] = 1948

# correct Tennessee State - Joined in 1986
ovc_df.loc[ovc_df['Institution'] == 'Tennessee State University', 'Joined'] = 1986

# add to list of dataframes
df_list.append(ovc_df)

# current
# former
# football
# ovc_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\2046602760.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
  ovc_df['Institution'] = ovc_df['Institution'].str.replace('\[.*\]', '')


# SKIP 

## Oklahoma Intercollegiate Conference (1914–1928)
The Oklahoma Intercollegiate Conference was an intercollegiate athletic conference that existed from 1914 to 1928 and the first of two conferences to share this name. The conference's members were located in the state of Oklahoma.[1] Some of its teams subsequently joined the Oklahoma Collegiate Conference, which eventually evolved into the second iteration of the Oklahoma Intercollegiate Conference in 1974.

## Oklahoma Collegiate Conference

The Oklahoma Collegiate Conference (OCC), also known as the Oklahoma Collegiate Athletic Conference (OCAC), was an intercollegiate athletic conference that existed from 1929 to 1974. The conference's members were located in the state of Oklahoma.[1] The league's predecessor was the first iteration of the Oklahoma Intercollegiate Conference, and its successor was the OIC's second iteration.

## Presidents' Athletic Conference

### History
The PAC was founded in 1955 by the presidents of Western Reserve University (1955–1967, operating athletically as Adelbert College from 1967 to 1970), Case Institute of Technology (1955–1970), John Carroll University (1955–1989) and Wayne State University (1955–1967).[1] Unlike other conferences at that time, the PAC was designed to be controlled by the presidents of the institutions rather than the athletic directors.[1] Member institutions were to admit athletes on the same academic standards as other students and award scholarships only based on academic achievement or need.[1]

By 1958, the PAC expanded east to include Allegheny College (1958–1984, 2022–present), Bethany College (1958–present), Thiel College (1958–present) and Washington & Jefferson College (1958–present).[1] Eventually, many other member institutions joined the PAC, like Chatham University (2007–present), Geneva College (2007–present), Grove City College (1984–present), Saint Vincent College (2006–present), Thomas More College (2005–2018), Waynesburg University (1990–present) and Westminster College (2000–present).[2]

Some former PAC member institutions include Alfred University (1996–1998), Carnegie Mellon University (1968–1990), Eastern Michigan University (1962–1967) and Hiram College (1971–1989).[2] On May 31, 2017, Thomas More College, now designated as a "University", announced its withdrawal from the PAC at the conclusion of the 2017–18 school year.[3]

In [31]:
# Presidents' Athletic Conference

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['PAC'])

# try to open tables with pandas
pac_table = pd.read_html(wiki_page.text)

current = pac_table[1]
former = pac_table[3]
# clean joined and left in former by only keeping 4 characters
former['Joined'] = former['Joined'].str[:4]
former['Left'] = former['Left'].str[:4]

# append to single dataframe
pac_df = pd.concat([current, former])

# add conference column
pac_df['Conference'] = 'Presidents Athletic Conf (PAC)'

# keep only the neseccesary columns
pac_df = pac_df[keep_cols]

# drop the citations from Joined column
pac_df['Joined'] = pac_df['Joined'].str.replace('\[.*\]', '')

## add to list of dataframes
df_list.append(pac_df)


# current
# former
# pac_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\4119179089.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
  pac_df['Joined'] = pac_df['Joined'].str.replace('\[.*\]', '')


## Rocky Mountain Athletic Conference

### History
Founded in 1909, the Rocky Mountain Athletic Conference is the fifth oldest active college athletic conference in the United States, the oldest in NCAA Division II, and the sixth to be founded after the Michigan Intercollegiate Athletic Association, the Big Ten Conference, the Southern Intercollegiate Athletic Association, the Ohio Athletic Conference, and the Missouri Valley Conference. For its first 30 years, the RMAC was considered a major conference, equivalent to today's NCAA Division I, before seven of its larger members left in 1938 to form the Mountain States Conference, also called the Skyline Conference.

The original name of Colorado Faculty Athletic Conference was changed to Rocky Mountain Faculty Athletic Conference (RMFAC) on May 7, 1910. The presidents assumed control of the league from the faculty in 1967 and changed the name to Rocky Mountain Athletic Conference. The Colorado Athletic Conference dissolved in 1996, with the RMAC absorbing the remaining CAC teams. The RMAC became an NCAA member in 1992 after competing in the NAIA through 1991.[1][2]

In [32]:
## Rocky Mountain Athletic Conference
### Important as a source of some big western programs (BYU - COlorado Colorado State, ect

# Can ignore the current members because they are all D2

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['RMAC'])

# try to open tables with pandas
rmac_table = pd.read_html(wiki_page.text)

# get the former members table
former = rmac_table[3]

# Drop Colorado College (will only cause problems)
former = former[former['Institution'] != 'Colorado College']

## PROBLEM ROWS
# Fort Hays State University[e] (1967-1972, 1989-2006)
# University of Nebraska at Kearney[g] (1989-1990, 1994-2012)
# Western New Mexico University (1967-1990, 2006-2016)

# Delete the orig rows
former = former[former['Institution'] != 'Fort Hays State University[e]']
former = former[former['Institution'] != 'University of Nebraska at Kearney[g]']
former = former[former['Institution'] != 'Western New Mexico University']

# create new rows
fort_hays1 = {'Institution': 'Fort Hays State University', 'Nickname': 'Tigers', 'Location': 'Hays, Kansas', 'Founded': 1902, 'Joined': 1967, 'Left': 1972, 'Enrollment': 15_000, 'Conference': 'Rocky Mountain Athletic'}
fort_hays2 = {'Institution': 'Fort Hays State University', 'Nickname': 'Tigers', 'Location': 'Hays, Kansas', 'Founded': 1902, 'Joined': 1989, 'Left': 2006, 'Enrollment': 15_000, 'Conference': 'Rocky Mountain Athletic'}
nebraska_kearney1 = {'Institution': 'University of Nebraska at Kearney', 'Nickname': 'Lopers', 'Location': 'Kearney, Nebraska', 'Founded': 1905, 'Joined': 1989, 'Left': 1990, 'Enrollment': 6_500, 'Conference': 'Rocky Mountain Athletic'}
nebraska_kearney2 = {'Institution': 'University of Nebraska at Kearney', 'Nickname': 'Lopers', 'Location': 'Kearney, Nebraska', 'Founded': 1905, 'Joined': 1994, 'Left': 2012, 'Enrollment': 6_500, 'Conference': 'Rocky Mountain Athletic'}
western_new_mexico1 = {'Institution': 'Western New Mexico University', 'Nickname': 'Mustangs', 'Location': 'Silver City, New Mexico', 'Founded': 1893, 'Joined': 1967, 'Left': 1990, 'Enrollment': 3_500, 'Conference': 'Rocky Mountain Athletic'}
western_new_mexico2 = {'Institution': 'Western New Mexico University', 'Nickname': 'Mustangs', 'Location': 'Silver City, New Mexico', 'Founded': 1893, 'Joined': 2006, 'Left': 2016, 'Enrollment': 3_500, 'Conference': 'Rocky Mountain Athletic'}

# list of rows
rmac_rows = [fort_hays1, fort_hays2, nebraska_kearney1, nebraska_kearney2, western_new_mexico1, western_new_mexico2]

# append rows to df
former = former.append(rmac_rows, ignore_index=True)

# add conference column
former['Conference'] = 'Rocky Mountain Athletic'
# keep only the neseccesary columns
rocky_mnt_df = former[keep_cols]

# Correct Wayne State College - name: Wayne State - Nebraska, Joined: 1989
rocky_mnt_df.loc[rocky_mnt_df['Institution'] == 'Wayne State College', 'Joined'] = 1989
rocky_mnt_df.loc[rocky_mnt_df['Institution'] == 'Wayne State College', 'Institution'] = 'Wayne State - Nebraska'

# add to list of dataframes
df_list.append(rocky_mnt_df)

# former
# rocky_mnt_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\1157041621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  former = former.append(rmac_rows, ignore_index=True)


# FCS INDEPENDENTS

Important to get this data

url = 

In [33]:
###########################
# NOTES - NOT ADDED TO FINAL DATAFRAME - NEEDS CHECKING

## FCS Independents -single table - need to parse the Years column for joined and left dates

url = 'https://en.wikipedia.org/wiki/NCAA_Division_I_FCS_independent_schools'

# get the tables from the wiki page
tables = pd.read_html(url)

# get the first table
current = tables[0]
# edit the current table

former = tables[1]

# deal with year column in former - if just one date then it is both joined and left, it might be a range ie 1987-1991
# if it is a range then the first date is joined and the second is left, split on the dash
former['Years'] = former['Years'].str.split('–')
# create new columns for joined and left
former['Joined'] = former['Years'].str[0]
former['Left'] = former['Years'].str[1]
# If Joined is Nan then copy value from Joined
former['Joined'] = former['Joined'].fillna(former['Left'])

# append to single dataframe
fcs_ind_df = pd.concat([current, former])

# create empty enrollment, founded and location columns
fcs_ind_df['Enrollment'] = ''
fcs_ind_df['Founded'] = ''
fcs_ind_df['Location'] = ''
# add conference column
fcs_ind_df['Conference'] = 'FCS Independent'

# Change Name to Institution
fcs_ind_df = fcs_ind_df.rename(columns={'Team': 'Institution'})

# add data to Kennesaw State - joined and left = 2023
fcs_ind_df.loc[fcs_ind_df['Institution'] == 'Kennesaw State', 'Joined'] = 2023
fcs_ind_df.loc[fcs_ind_df['Institution'] == 'Kennesaw State', 'Left'] = 2023

# If Left is Nan then copy value from Joined
fcs_ind_df['Left'] = fcs_ind_df['Left'].fillna(fcs_ind_df['Joined'])

# keep only the neseccesary columns
fcs_ind_df = fcs_ind_df[keep_cols]



# fcs_ind_df
# former
# current

## South Atlantic Intercollegiate Athletic Association

#### Only SPonsered Football from 1912 to 1921 so 

In [34]:
# South Atlantic Intercollegiate Athletic Association
#### Only SPonsered Football from 1912 to 1921 so all other years are not needed

# Membership
# Catholic 1916–1921[6][7]
# Davidson 1916–1921[6][7]
# George Washington 1916–1921[6][8][7]
# Georgetown 1911–1921[6][9]
# Johns Hopkins 1911–1921[8][9]
# Maryland 1916–1921[6][10][7]
# North Carolina 1911–1921[6][9]
# North Carolina State 1911–1921[8][9]
# Richmond 1911–1921[8][11]
# St. John's (MD) 1921[8]
# Trinity College 1916–1921[8][7]
# Virginia 1911–1921[6][9]
# Virginia Polytechnic Institute 1911–1921[6][9]
# Virginia Military Institute 1916–1921[6][7]
# Washington and Lee 1911–1921[8][9][12]
# William & Mary 1916–1921[8][7]


# create rows manually
catholic = {'Institution': 'Catholic University of America', 'Nickname': 'Cardinals', 'Location': 'Washington, D.C.', 'Founded': 1887, 'Joined': 1916, 'Left': 1921, 'Enrollment': 7_000, 'Conference': 'South Atlantic Intercollegiate Athletic'}
davidson = {'Institution': 'Davidson College', 'Nickname': 'Wildcats', 'Location': 'Davidson, North Carolina', 'Founded': 1837, 'Joined': 1916, 'Left': 1921, 'Enrollment': 1_800, 'Conference': 'South Atlantic Intercollegiate Athletic'}
george_washington = {'Institution': 'George Washington University', 'Nickname': 'Colonials', 'Location': 'Washington, D.C.', 'Founded': 1821, 'Joined': 1916, 'Left': 1921, 'Enrollment': 28_000, 'Conference': 'South Atlantic Intercollegiate Athletic'}
georgetown = {'Institution': 'Georgetown University', 'Nickname': 'Hoyas', 'Location': 'Washington, D.C.', 'Founded': 1789, 'Joined': 1911, 'Left': 1921, 'Enrollment': 19_000, 'Conference': 'South Atlantic Intercollegiate Athletic'}
johns_hopkins = {'Institution': 'Johns Hopkins University', 'Nickname': 'Blue Jays', 'Location': 'Baltimore, Maryland', 'Founded': 1876, 'Joined': 1911, 'Left': 1921, 'Enrollment': 24_000, 'Conference': 'South Atlantic Intercollegiate Athletic'}
maryland = {'Institution': 'University of Maryland', 'Nickname': 'Terrapins', 'Location': 'College Park, Maryland', 'Founded': 1856, 'Joined': 1916, 'Left': 1921, 'Enrollment': 41_200, 'Conference': 'South Atlantic Intercollegiate Athletic'}
north_carolina = {'Institution': 'University of North Carolina', 'Nickname': 'Tar Heels', 'Location': 'Chapel Hill, North Carolina', 'Founded': 1789, 'Joined': 1911, 'Left': 1921, 'Enrollment': 30_000, 'Conference': 'South Atlantic Intercollegiate Athletic'}
north_carolina_state = {'Institution': 'North Carolina State University', 'Nickname': 'Wolfpack', 'Location': 'Raleigh, North Carolina', 'Founded': 1887, 'Joined': 1911, 'Left': 1921, 'Enrollment': 36_000, 'Conference': 'South Atlantic Intercollegiate Athletic'}
richmond = {'Institution': 'University of Richmond', 'Nickname': 'Spiders', 'Location': 'Richmond, Virginia', 'Founded': 1830, 'Joined': 1911, 'Left': 1921, 'Enrollment': 4_200, 'Conference': 'South Atlantic Intercollegiate Athletic'}
st_johns = {'Institution': 'St. John\'s College (Maryland)', 'Nickname': 'Johnnies', 'Location': 'Annapolis, Maryland', 'Founded': 1696, 'Joined': 1921, 'Left': 1921, 'Enrollment': 500, 'Conference': 'South Atlantic Intercollegiate Athletic'}
trinity = {'Institution': 'Trinity College (North Carolina)', 'Nickname': 'Tigers', 'Location': 'Durham, North Carolina', 'Founded': 1838, 'Joined': 1916, 'Left': 1921, 'Enrollment': 6_000, 'Conference': 'South Atlantic Intercollegiate Athletic'}
virginia = {'Institution': 'University of Virginia', 'Nickname': 'Cavaliers', 'Location': 'Charlottesville, Virginia', 'Founded': 1819, 'Joined': 1911, 'Left': 1921, 'Enrollment': 24_000, 'Conference': 'South Atlantic Intercollegiate Athletic'}
virginia_polytechnic = {'Institution': 'Virginia Polytechnic Institute', 'Nickname': 'Hokies', 'Location': 'Blacksburg, Virginia', 'Founded': 1872, 'Joined': 1911, 'Left': 1921, 'Enrollment': 34_000, 'Conference': 'South Atlantic Intercollegiate Athletic'}
virginia_military = {'Institution': 'Virginia Military Institute', 'Nickname': 'Keydets', 'Location': 'Lexington, Virginia', 'Founded': 1839, 'Joined': 1916, 'Left': 1921, 'Enrollment': 1_700, 'Conference': 'South Atlantic Intercollegiate Athletic'}
washington_lee = {'Institution': 'Washington and Lee University', 'Nickname': 'Generals', 'Location': 'Lexington, Virginia', 'Founded': 1749, 'Joined': 1911, 'Left': 1921, 'Enrollment': 1_900, 'Conference': 'South Atlantic Intercollegiate Athletic'}
william_mary = {'Institution': 'College of William & Mary', 'Nickname': 'Tribe', 'Location': 'Williamsburg, Virginia', 'Founded': 1693, 'Joined': 1916, 'Left': 1921, 'Enrollment': 8_600, 'Conference': 'South Atlantic Intercollegiate Athletic'}

# create list of rows
saiac_rows = [catholic, davidson, george_washington, georgetown, johns_hopkins, maryland, north_carolina, north_carolina_state, richmond, st_johns, trinity, virginia, virginia_polytechnic, virginia_military, washington_lee, william_mary]

# create blank dataframe
saiac_df = pd.DataFrame(columns=['Institution', 'Nickname', 'Location', 'Founded', 
           'Joined', 'Left', 'Enrollment', 'Conference'])

# append rows to df
saiac_df = saiac_df.append(saiac_rows, ignore_index=True)

# add to list of dataframes
df_list.append(saiac_df)

# saiac_df


C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\17348821.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  saiac_df = saiac_df.append(saiac_rows, ignore_index=True)


## Southern California Intercollegiate Athletic Conference

A forerunner conference to the SCIAC was the Intercollegiate Football Association of Southern California, which existed in the 1890s. It included Occidental, Caltech (then called Throop Polytechnic), USC, Chaffey College and Los Angeles High School.

The SCIAC was founded in 1915 with five member schools with the goals to promote amateurism in athletics. The five founding members, all of whom are still members, are Throop College of Technology (now California Institute of Technology), Occidental College, Pomona College, the University of Redlands, and Whittier College. Although all five original charter members are still affiliated with the SCIAC, only two, Occidental and Redlands, have had uninterrupted membership. The acronym SCIAC (standing for Southern California Interscholastic Athletic Council) was in use during 1913 and 1914 until that organization became the CIF Southern Section.[1]

In [35]:
## Southern California Intercollegiate Athletic Conference

# Only interested in former members because that includes UCLA
### EVENTUALLY MIGHT WANT TO GET THE CURRENT MEMBERS BECAUSE SOME GO WAY BACK AND MAY BE IN GAME DATA FROM EARLY 20th

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['SCIAC'])

# try to open tables with pandas
sciac_table = pd.read_html(wiki_page.text)

# get the former members table

former = sciac_table[2]

# keep only first 4 characters of Joined and Left
former['Joined'] = former['Joined'].str[:4]
former['Left'] = former['Left'].str[:4]
# add conference column
former['Conference'] = 'Southern California Intercollegiate Athletic'
# Rename Location[a] to Location
former = former.rename(columns={'Location[a]': 'Location'})

# Rename Instituations
# former.loc[former['Institution'] == 'California Institute of Technology', 'Institution'] = 'California Tech'
former.loc[former['Institution'] == 'University of California, Los Angeles(UCLA)', 'Institution'] = 'UCLA'
# drop 'University of'
former['Institution'] = former['Institution'].str.replace('University of ', '')
# Rename 'California, Santa Barbara(UC Santa Barbara, UCSB)' to 'California, Santa Barbara'
former.loc[former['Institution'] == 'California, Santa Barbara(UC Santa Barbara, UCSB)', 'Institution'] = 'California - Santa Barbara'

sciac_df = former
# keep only the neseccesary columns
sciac_df = former[keep_cols]

# add to list of dataframes
df_list.append(sciac_df)

# former
# sciac_df

## Southern Intercollegiate Athletic Association

The Southern Intercollegiate Athletic Association (SIAA) was one of the first collegiate athletic conferences in the United States. Twenty-seven of the current Division I FBS (formerly Division I-A) football programs were members of this conference at some point, as were at least 19 other schools. Every member of the current Southeastern Conference except Arkansas and Missouri, as well as six of the 15 current members of the Atlantic Coast Conference plus future SEC member University of Texas at Austin, currently of the Big 12 Conference (and previously of the now defunct Southwest Conference), formerly held membership in the SIAA.

In [36]:
## Southern Intercollegiate Athletic Association - SIAA

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['SIAA'])

# try to open tables with pandas
siaa_table = pd.read_html(wiki_page.text)

# get the former members table
former = siaa_table[3]
# Combine city and state into a single Location column with a comma between
former['Location'] = former['City'] + ', ' + former['State']
# drop City and State columns
former = former.drop(columns=['City', 'State'])

# add conference column
former['Conference'] = 'Southern Intercollegiate Athletic Association'
# add Empty Enrollment, Nickname and Founded columns
former['Nickname'] = ''
former['Enrollment'] = ''
former['Founded'] = ''
# rename School to Institution
former = former.rename(columns={'School': 'Institution'})


#### CHAT GPT CODE TO PARSE THE Tenure COLUMN

import re
# Function to extract year ranges from the tenure string
def extract_year_ranges(tenure_str):
    clean_str = re.sub(r'\[.*?\]', '', tenure_str)
    year_ranges = clean_str.replace(';', ',').split(',')
    year_ranges = [yr.strip() for yr in year_ranges]
    return year_ranges

# Function to split the 'Tenure' into 'Joined' and 'Left' columns
def split_tenure(row):
    tenure_str = row['Tenure']
    if '–' in tenure_str:
        # Handle cases with more than one '–'
        if tenure_str.count('–') > 1:
            # Use the first '–' to split
            joined, left = tenure_str.split('–', 1)
        else:
            joined, left = tenure_str.split('–')
    else:
        joined = left = re.sub(r'Part of ', '', tenure_str)
    row['Joined'] = joined
    row['Left'] = left
    return row

# Function to transform the DataFrame
def transform_dataframe(df):
    # Expand rows based on year ranges in 'Tenure'
    new_rows = []
    for index, row in df.iterrows():
        year_ranges = extract_year_ranges(row['Tenure'])
        for yr in year_ranges:
            new_row = row.copy()
            new_row['Tenure'] = yr
            new_rows.append(new_row)
    df_expanded = pd.DataFrame(new_rows).reset_index(drop=True)
    
    # Split 'Tenure' into 'Joined' and 'Left'
    df_expanded = df_expanded.apply(split_tenure, axis=1)
    df_expanded.drop(columns=['Tenure'], inplace=True)
    
    return df_expanded

# Transform the DataFrame

siaa_df = transform_dataframe(former)

# Show the first few rows of the transformed DataFrame






# keep only the neseccesary columns
siaa_df = siaa_df[keep_cols]

# former
# siaa_df

## The Southland Conference

The Southland Conference, abbreviated as SLC, is a collegiate athletic conference which operates in the South Central United States (specifically Texas and Louisiana). It participates in the NCAA's Division I for all sports; for football, it participates in the Division I Football Championship Subdivision (FCS). The Southland sponsors 18 sports, 10 for women and eight for men, and is governed by a presidential Board of Directors and an Advisory Council of athletic and academic administrators. Chris Grant became the Southland's seventh commissioner on April 5, 2022.[1] From 1996 to 2002, for football only, the Southland Conference was known as the Southland Football League.[2]

Founded in 1963, its members were Abilene Christian College (now Abilene Christian University; departed in 1973 for NCAA Division II, but moved to Division I and rejoined the Southland in 2013), Arkansas State College (now Arkansas State University; departed in 1987, now a member of the Sun Belt Conference), Arlington State College (now The University of Texas at Arlington, departed in 2012 and now in the Western Athletic Conference),[5] Lamar State College of Technology (now Lamar University; departed in 1987, rejoined in 1999, left again in 2021, returned in 2022),[6] and Trinity University (departed in 1972, now participating in NCAA Division III).

In [37]:
## Southland Conference

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['Southland'])

# try to open tables with pandas
southland_table = pd.read_html(wiki_page.text)

current = southland_table[1]
former = southland_table[3]

# Fix issues with current members
# keep only first four characters of Joined
current['Joined'] = current['Joined'].str[:4]

# append to single dataframe
southland_df = pd.concat([current, former])

# remove citations from Institution column
southland_df['Institution'] = southland_df['Institution'].str.replace('\[.*\]', '')

# add conference column
southland_df['Conference'] = 'Southland'
# drop citations from Enrollment column
southland_df['Enrollment'] = southland_df['Enrollment'].str.replace('\[.*\]', '')
# drop commas from Enrollment column
southland_df['Enrollment'] = southland_df['Enrollment'].str.replace(',', '')
# keep only the neseccesary columns
southland_df = southland_df[keep_cols]

# remove any citations from Joined column
southland_df['Joined'] = southland_df['Joined'].str.replace('\[.*\]', '')

# NEED TO CORRECT
# Lamar Univeristy - (1963-1987, 1999-2021, 2022-present)
# Abilene Christian University - (1963-1973, 2013-2021)
# drop the orig rows
southland_df = southland_df[southland_df['Institution'] != 'Lamar University']
southland_df = southland_df[southland_df['Institution'] != 'Abilene Christian University']
# create new rows
lamar1 = {'Institution': 'Lamar University', 'Nickname': 'Cardinals', 'Location': 'Beaumont, Texas', 'Founded': 1923, 'Joined': 1963, 'Left': 1987, 'Enrollment': 15_000, 'Conference': 'Southland'}
lamar2 = {'Institution': 'Lamar University', 'Nickname': 'Cardinals', 'Location': 'Beaumont, Texas', 'Founded': 1923, 'Joined': 1999, 'Left': 2021, 'Enrollment': 15_000, 'Conference': 'Southland'}
lamar3 = {'Institution': 'Lamar University', 'Nickname': 'Cardinals', 'Location': 'Beaumont, Texas', 'Founded': 1923, 'Joined': 2022, 'Left': '', 'Enrollment': 15_000, 'Conference': 'Southland'}
abilene1 = {'Institution': 'Abilene Christian University', 'Nickname': 'Wildcats', 'Location': 'Abilene, Texas', 'Founded': 1906, 'Joined': 1963, 'Left': 1973, 'Enrollment': 5_000, 'Conference': 'Southland'}
abilene2 = {'Institution': 'Abilene Christian University', 'Nickname': 'Wildcats', 'Location': 'Abilene, Texas', 'Founded': 1906, 'Joined': 2013, 'Left': 2021, 'Enrollment': 5_000, 'Conference': 'Southland'}

# list of rows
southland_rows = [lamar1, lamar2, lamar3, abilene1, abilene2]

# add rows to df
southland_df = southland_df.append(southland_rows, ignore_index=True)

# add to list of dataframes
df_list.append(southland_df)


# current
# former
# southland_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3814077254.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  southland_df['Institution'] = southland_df['Institution'].str.replace('\[.*\]', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3814077254.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
  southland_df['Enrollment'] = southland_df['Enrollment'].str.replace('\[.*\]', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3814077254.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  southland_df['Joined'] = southland_df['Joined'].str.replace('\[.*\]', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3814077254.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  southland_df = southland_df.append(southland_rows, ignore_index=T

## Texas Intercollegiate Athletic Association

The Texas Intercollegiate Athletic Association (TIAA) was a college sports association that operated from 1909 to 1932. All of its members were located in the US state of Texas.

### History
Founded in 1909 by Southwestern University, Austin College, Texas Christian University, Texas, Texas A&M, Baylor University and Trinity University the TIAA had a changing set of members that spun off into the Southwest Conference, Lone Star Conference and the Texas Conference. The league had been formed to rid college athletics of objectionable elements like gambling and place them entirely under the control of the schools. At first the league worked well, but soon the disparity in the sizes of the schools became an issue. The large state schools, with bigger stadiums and crowds, began to refuse to travel to the smaller schools and insisted on playing that at home.[1]

This battle between the large and small schools led to the first big change in 1914, when Texas, A&M, Baylor and Southwestern left to form the Southwest Conference, with some considering themselves to be members of both. After two seasons, Southwestern left the Southwest conference and returned to the TIAA. The conferences became completely separate in 1917. In 1922, TIAA expanded by including North Texas.[2] In 1923, TCU left to join the Southwest Conference and in 1924, Rice followed suit.

In 1925, the TIAA was split between members who wanted to allow freshmen and transfers to play and schools that did not. This fault line also separated the teachers colleges that had joined over the years and the church-sponsored schools that had been founders. In May 1925, those that wanted to allow them to play—Trinity, Simmons University (now Hardin–Simmons University), Austin College, Howard Payne, and Southwestern—left to form the Texas Conference, but they agreed to play out the fall 1925 football season within the TIAA. Play within the Texas Conference began with the 1926 basketball season. The five departing schools were all church supported and agreed to ally themselves "along denominational lines."[2][3]

By 1930, the TIAA had expanded back to 11 members. This included teachers colleges such as Sul Ross, West Texas, East Texas, North Texas, Sam Houston, Stephen F. Austin, and Southwest Texas as well as Abilene Christian College, Daniel Baker, Texas A&I and McMurry. But in May 1931, five TIAA schools joined together to form the Lone Star Conference. These were Sam Houston, Stephen F. Austin, East Texas, Southwest Texas and North Texas.[4][2] As a result, the conference was dissolved in 1932. Many of the members became independents and eventually wound up in the short-lived Alamo Conference.

In [38]:
# Texas Intercollegiate Athletic Association (1909 TO 1932)

# Membership
all_members = ['Southwestern University', 'Austin College', 'Texas Christian University', 'Texas', 
               'Texas A&M', 'Baylor', 'Trinity College', 'Rice', 'Texas Mines', 'Texas Tech']

# Left in 1914 - Texas, Texas A&M, Baylor, Southwestern left to form the Southwest Conference
#  North Texas  joined in 1922
# Texas Christian University left in 1923
# 1924  Rice left as well
# The rest of the schools left in 1925

# create rows manually
southwestern = {'Institution': 'Southwestern University', 'Nickname': 'Pirates', 'Location': 'Georgetown, Texas', 'Founded': 1840, 'Joined': 1909, 'Left': 1914, 'Enrollment': 1_500, 'Conference': 'Texas Intercollegiate Athletic'}
austin = {'Institution': 'Austin College', 'Nickname': 'Kangaroos', 'Location': 'Sherman, Texas', 'Founded': 1849, 'Joined': 1909, 'Left': 1925, 'Enrollment': 1_300, 'Conference': 'Texas Intercollegiate Athletic'}
texas_christian = {'Institution': 'Texas Christian University', 'Nickname': 'Horned Frogs', 'Location': 'Fort Worth, Texas', 'Founded': 1873, 'Joined': 1909, 'Left': 1923, 'Enrollment': 11_000, 'Conference': 'Texas Intercollegiate Athletic'}
texas = {'Institution': 'University of Texas', 'Nickname': 'Longhorns', 'Location': 'Austin, Texas', 'Founded': 1883, 'Joined': 1909, 'Left': 1914, 'Enrollment': 51_000, 'Conference': 'Texas Intercollegiate Athletic'}
texas_am = {'Institution': 'Texas A&M University', 'Nickname': 'Aggies', 'Location': 'College Station, Texas', 'Founded': 1876, 'Joined': 1909, 'Left': 1914, 'Enrollment': 69_000, 'Conference': 'Texas Intercollegiate Athletic'}
baylor = {'Institution': 'Baylor University', 'Nickname': 'Bears', 'Location': 'Waco, Texas', 'Founded': 1845, 'Joined': 1909, 'Left': 1914, 'Enrollment': 17_000, 'Conference': 'Texas Intercollegiate Athletic'}
trinity = {'Institution': 'Trinity University', 'Nickname': 'Tigers', 'Location': 'San Antonio, Texas', 'Founded': 1869, 'Joined': 1909, 'Left': 1925, 'Enrollment': 2_500, 'Conference': 'Texas Intercollegiate Athletic'}
rice = {'Institution': 'Rice University', 'Nickname': 'Owls', 'Location': 'Houston, Texas', 'Founded': 1912, 'Joined': 1909, 'Left': 1924, 'Enrollment': 7_000, 'Conference': 'Texas Intercollegiate Athletic'}
texas_mines = {'Institution': 'University of Texas at El Paso', 'Nickname': 'Miners', 'Location': 'El Paso, Texas', 'Founded': 1914, 'Joined': 1914, 'Left': 1925, 'Enrollment': 25_000, 'Conference': 'Texas Intercollegiate Athletic'}
texas_tech = {'Institution': 'Texas Tech University', 'Nickname': 'Red Raiders', 'Location': 'Lubbock, Texas', 'Founded': 1923, 'Joined': 1925, 'Left': 1925, 'Enrollment': 40_000, 'Conference': 'Texas Intercollegiate Athletic'}


# create list of rows
tiaa_rows = [southwestern, austin, texas_christian, texas, texas_am, baylor, trinity, rice, texas_mines, texas_tech]

# create blank dataframe
texas_iaa_df = pd.DataFrame(columns=['Institution', 'Nickname', 'Location', 'Founded',
                                    'Joined', 'Left', 'Enrollment', 'Conference'])

# append rows to df
texas_iaa_df = texas_iaa_df.append(tiaa_rows, ignore_index=True)

# Correct the Southwestern University - two rows (1909-1914, 1917-1925)
# drop the orig row
texas_iaa_df = texas_iaa_df[texas_iaa_df['Institution'] != 'Southwestern University']
# add new rows
southwestern1 = {'Institution': 'Southwestern University', 'Nickname': 'Pirates', 'Location': 'Georgetown, Texas', 'Founded': 1840, 'Joined': 1909, 'Left': 1914, 'Enrollment': 1_500, 'Conference': 'Texas Intercollegiate Athletic'}
southwestern2 = {'Institution': 'Southwestern University', 'Nickname': 'Pirates', 'Location': 'Georgetown, Texas', 'Founded': 1840, 'Joined': 1917, 'Left': 1925, 'Enrollment': 1_500, 'Conference': 'Texas Intercollegiate Athletic'}

# add rows to df
texas_iaa_df = texas_iaa_df.append(southwestern1, ignore_index=True)

## LATER VERSION

# By 1930, the TIAA had expanded back to 11 members. This included teachers colleges such as Sul Ross, West Texas, East Texas, 
# North Texas, Sam Houston, Stephen F. Austin, and Southwest Texas as well as Abilene Christian College, 
# Daniel Baker, Texas A&I and McMurry. But in May 1931, five TIAA schools joined together to form the Lone Star Conference. 
# These were Sam Houston, Stephen F. Austin, East Texas, Southwest Texas and North Texas.[4][2] 
# As a result, the conference was dissolved in 1932. Many of the members became independents and eventually 
# wound up in the short-lived Alamo Conference.

# create those rows manually

sul_ross = {'Institution': 'Sul Ross State University', 'Nickname': 'Lobos', 'Location': 'Alpine, Texas', 'Founded': 1917, 'Joined': 1930, 'Left': 1932, 'Enrollment': 2_000, 'Conference': 'Texas Intercollegiate Athletic'}
west_texas = {'Institution': 'West Texas A&M University', 'Nickname': 'Buffaloes', 'Location': 'Canyon, Texas', 'Founded': 1910, 'Joined': 1930, 'Left': 1932, 'Enrollment': 10_000, 'Conference': 'Texas Intercollegiate Athletic'}
east_texas = {'Institution': 'Texas A&M University–Commerce', 'Nickname': 'Lions', 'Location': 'Commerce, Texas', 'Founded': 1889, 'Joined': 1930, 'Left': 1932, 'Enrollment': 12_000, 'Conference': 'Texas Intercollegiate Athletic'}
north_texas = {'Institution': 'University of North Texas', 'Nickname': 'Mean Green', 'Location': 'Denton, Texas', 'Founded': 1890, 'Joined': 1930, 'Left': 1932, 'Enrollment': 40_000, 'Conference': 'Texas Intercollegiate Athletic'}
sam_houston = {'Institution': 'Sam Houston State University', 'Nickname': 'Bearkats', 'Location': 'Huntsville, Texas', 'Founded': 1879, 'Joined': 1930, 'Left': 1932, 'Enrollment': 21_000, 'Conference': 'Texas Intercollegiate Athletic'}
stephen_austin = {'Institution': 'Stephen F. Austin State University', 'Nickname': 'Lumberjacks', 'Location': 'Nacogdoches, Texas', 'Founded': 1923, 'Joined': 1930, 'Left': 1932, 'Enrollment': 13_000, 'Conference': 'Texas Intercollegiate Athletic'}
southwest_texas = {'Institution': 'Texas State University', 'Nickname': 'Bobcats', 'Location': 'San Marcos, Texas', 'Founded': 1899, 'Joined': 1930, 'Left': 1932, 'Enrollment': 38_000, 'Conference': 'Texas Intercollegiate Athletic'}

# create list of rows
tiaa_rows = [sul_ross, west_texas, east_texas, north_texas, sam_houston, stephen_austin, southwest_texas]

# append rows to df
texas_iaa_df = texas_iaa_df.append(tiaa_rows, ignore_index=True)

# add to list of dataframes
df_list.append(texas_iaa_df)


# texas_iaa_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3035612779.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texas_iaa_df = texas_iaa_df.append(tiaa_rows, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3035612779.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texas_iaa_df = texas_iaa_df.append(southwestern1, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3035612779.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texas_iaa_df = texas_iaa_df.append(tiaa_rows, ignore_index=True)


# Virginia College Athletic Association

The Virginia College Athletic Association (VCAA), also known as the Virginia Collegiate Athletic Association, was a short-lived intercollegiate athletic conference that existed from 1972 to 1975. As its name suggests, the league's members were located in the state of Virginia.[1] The present-day Old Dominion Athletic Conference subsequently began play in 1976.

In [39]:
### SKIP IT

### Virginia College Athletic Association

## Volunteer State Athletic Conference

### History
The VSAC was organized in the 1940s and dissolved in the early 1980s. Member schools were in the National Association of Intercollegiate Athletics (NAIA). Long-term members of the conference included the institutions now known as Belmont University, Bethel University, Bryan College, Carson–Newman University, Christian Brothers University, King University, Lee University, Lincoln Memorial University, Lipscomb University, Milligan College, Tusculum University, Tennessee Wesleyan University, and Union University, as well as the now-defunct Lambuth University.

### Dissolution
The conference dissolved when the institutions in the eastern portion of the state seceded to form the Tennessee Valley Athletic Conference (TVAC). Those in the western part of the state formed in turn the Tennessee Collegiate Athletic Conference (TCAC). The Appalachian Athletic Conference is the direct successor of the TVAC, and was known as such until the mid-1990s when the addition of schools in Virginia, Kentucky, and North Carolina necessitated the name change. The TranSouth Athletic Conference, which operated from 1996 to 2013, is sometimes regarded as something of a successor to the TCAC. Sponsorship of football by the conference ended after the 1962 season, over two decades before the conference dissolved with regard to its other sports.

Other founding members of the VSAC included East Tennessee State University, Middle Tennessee State University, Austin Peay State University, and Tennessee Technological University; these schools all left the VSAC to join the Ohio Valley Conference. As of 2019, Austin Peay and Tennessee Tech remain in the OVC, while East Tennessee State is in the Southern Conference and Middle Tennessee in Conference USA.

In [40]:
### SKIP IT

# Western Interstate University Football Association

The Western Interstate University Football Association (WIUFA) was one of the first intercollegiate athletic conferences in the United States,[1] existing from 1892 to 1897.

### Formation, history and evolution
The football teams from the Universities of Iowa, Kansas, Missouri, and Nebraska formed the conference and competed against each other annually. Early WIUFA play led to the transition of the famous rivalry between Kansas and Missouri to the football field as many of the fans and some of the first players on both teams were the sons of men who had fought each other on either side of the conflict in Bleeding Kansas and later the Civil War. Racial tension surrounding the participation of Iowa's Frank Kinney Holbrook in the 1896 game between Iowa and Missouri ended up preventing what may have become a long-standing rivalry between the two schools.[2][3]

All four members of the WIUFA would later be among the original members of the Missouri Valley Intercollegiate Athletic Association (MVIAA), which was formed in 1907. Kansas, Missouri and Nebraska stayed with the MVIAA as it evolved over the decades into the Big Eight Conference, which existed through the 1995 season, while Iowa remained a member of the Big Ten Conference and left after only one season of MVIAA competition. Beginning in 1996, the members of the Big Eight joined with four schools of the former Southwest Conference to form the new Big 12 Conference.

### Record-keeping discrepancies
For every year of the conference's existence, except 1896, there was at least one disagreement as to the final score of a game between member schools. However, in no cases is the final outcome of any game disputed.[4]

In [41]:
# Western Interstate University Football Association 
# only had 4 members and only existed from 1892- 1897
# create rows manually

# Membership -  Kansas, Nebraska, Missouri, Iowa
# all joined in 1982 and left in 1897

# create new dataframe
western_iufa_df = pd.DataFrame(columns=['Institution', 'Nickname', 'Location', 'Founded',
                                    'Joined', 'Left', 'Enrollment', 'Conference'])

# create rows manually
kansas = {'Institution': 'University of Kansas', 'Nickname': 'Jayhawks', 'Location': 'Lawrence, Kansas', 'Founded': 1865, 'Joined': 1892, 'Left': 1897, 'Enrollment': 28_000, 'Conference': 'Western Interstate University Football Association'}
nebraska = {'Institution': 'University of Nebraska', 'Nickname': 'Cornhuskers', 'Location': 'Lincoln, Nebraska', 'Founded': 1869, 'Joined': 1892, 'Left': 1897, 'Enrollment': 26_000, 'Conference': 'Western Interstate University Football Association'}
missouri = {'Institution': 'University of Missouri', 'Nickname': 'Tigers', 'Location': 'Columbia, Missouri', 'Founded': 1839, 'Joined': 1892, 'Left': 1897, 'Enrollment': 30_000, 'Conference': 'Western Interstate University Football Association'}
iowa = {'Institution': 'University of Iowa', 'Nickname': 'Hawkeyes', 'Location': 'Iowa City, Iowa', 'Founded': 1847, 'Joined': 1892, 'Left': 1897, 'Enrollment': 32_000, 'Conference': 'Western Interstate University Football Association'}

# create list of rows
wiufa_rows = [kansas, nebraska, missouri, iowa]

# append rows to df
western_iufa_df = western_iufa_df.append(wiufa_rows, ignore_index=True)

# add to list of dataframes
df_list.append(western_iufa_df)

# western_iufa_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\429297056.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  western_iufa_df = western_iufa_df.append(wiufa_rows, ignore_index=True)


## West Virginia Intercollegiate Athletic Conference

The West Virginia Intercollegiate Athletic Conference (WVIAC) was a collegiate athletic conference which historically operated exclusively in the state of West Virginia, but briefly had one Kentucky member in its early years, and expanded into Pennsylvania in its final years. It participated in the Division II ranks of the National Collegiate Athletic Association (NCAA), originally affiliated in the National Association of Intercollegiate Athletics (NAIA) until 1994, but held its final athletic competitions in spring 2013, and officially disbanded on September 1 of that year. Its football-playing members announced in June 2012 that they planned to withdraw to form a new Division II conference at the end of the 2012–13 season; this led to a chain of conference moves that saw all but one of the WVIAC's members find new conference homes.

History
The conference was rated as one of the oldest in intercollegiate athletics, dating back to its founding in 1924 by the West Virginia Department of Education.

In its final school year of 2012–13, the WVIAC offered championships in 16 sports and was headquartered in Princeton, West Virginia. Men's championships were offered in football, basketball, baseball, track, cross country, soccer, tennis, and golf. Women's titles were contested in volleyball, softball, basketball, cross country, soccer, track, tennis, and golf.

The WVIAC moved into the NCAA Division II in 1994 after its long affiliation with the NAIA.

Its post-season basketball tournament, which was first conducted in 1936, was at the time of the conference's demise one of the oldest college post-season tournaments in continuous existence—only the Southern Conference men's basketball tournament, established in 1922, was older.

In [42]:
## West Virginia Intercollegiate Athletic Conference

# only need to worry about former members because current members aren't in the relevant

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['WVIAC'])

# try to open tables with pandas
wviac_table = pd.read_html(wiki_page.text)

# get the former members table
former = wviac_table[2]
# remove citations from Institution column
former['Institution[a]'] = former['Institution[a]'].str.replace('\[.*\]', '')
# reanme Institution[a] to Institution
former = former.rename(columns={'Institution[a]': 'Institution'})

# add conference column
former['Conference'] = 'West Virginia Intercollegiate Athletic'
# add Empty Enrollment and Nickname columns
former['Nickname'] = ''
former['Enrollment'] = ''

# keep only the neseccesary columns
wviac_df = former[keep_cols]


# add to list of dataframes
df_list.append(wviac_df)

# former
# wviac_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3137597294.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  former['Institution[a]'] = former['Institution[a]'].str.replace('\[.*\]', '')


## Yankee Conference

The Yankee Conference was a collegiate sports conference in the eastern United States. From 1947 to 1976, it sponsored competition in many sports, but was a football-only league from mid-1976 until its dissolution in 1996. It is essentially the ancestor of today's CAA Football, the legally separate football league operated by the Coastal Athletic Association (CAA), and the continuation of the New England Conference, though all three leagues were founded under different charters and are considered separate conferences by the NCAA. Also, CAA Football does not recognize the New England Conference as one of its predecessors, though it does recognize the Yankee Conference as such.[1]

For the first half of its history, the Yankee Conference consisted of the flagship public universities of the six New England states. Conference expansion in the 1980s and 1990s added several colleges and universities from the Mid-Atlantic region.

### Formation
In 1945, Northeastern University, the only private school in the New England Conference, announced its departure.[citation needed] A committee formed by the remaining four members, land-grant colleges and universities representing Connecticut, Maine, New Hampshire and Rhode Island, recommended that they join with the other two New England land-grant institutions, Massachusetts State College (which had also been a founding member of the NEC in 1923, but left in the 1930s) and the University of Vermont, in a new athletics league. This led to the formation of the Yankee Conference in December 1946,[2] with athletic competition beginning in the 1947–48 school year.

In [43]:
## Yankee Conference

# call the wiki page using dictionary
wiki_page = requests.get(conf_dict['Yankee'])

# try to open tables with pandas
yankee_table = pd.read_html(wiki_page.text)

# get the members table
current = yankee_table[1]
# remove any non digits from Joined
current['Joined'] = current['Joined'].str.replace('\D', '')

# Teams that left in 1996 - Connecticut, Maine, Massachusetts, New Hampshire, Rhode Island, Vermont
# left in 1972 - Holy Cross

# all others left in 1996 - Boston University, Delaware, Richmond, Villanova, James Madison, Northeastern, William & Mary

# create the left coulm with values described as above
left_96 = ['University of Connecticut', 'University of Maine', 'University of Massachusetts Amherst',
           'University of New Hampshire', 'University of Rhode Island', 'University of Vermont']

left_72 = ['College of the Holy Cross']

left_96_all = ['Boston University', 'University of Delaware', 'University of Richmond', 'Villanova University',
                'James Madison University', 'Northeastern University', 'College of William & Mary']

# create new column for left based on the lists above
current['Left'] = ''
current.loc[current['Institution'].isin(left_96), 'Left'] = 1996
current.loc[current['Institution'].isin(left_72), 'Left'] = 1972
current.loc[current['Institution'].isin(left_96_all), 'Left'] = 1996

# add conference column
current['Conference'] = 'Yankee'

# add empty columns for Enrollment
current['Enrollment'] = ''

# keep only the neseccesary columns
yankee_df = current[keep_cols]

# add to list of dataframes
df_list.append(yankee_df)


# current
# yankee_df

C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\814404202.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  current['Joined'] = current['Joined'].str.replace('\D', '')


# LAST CONFERRENCE ON THE LIST !!!!!!!

In [44]:
## use the list of dataframe to make a single dataframe - name it MINOR_CONF_DF

# create empty dataframe
minor_conf_df = pd.DataFrame(columns=['Institution', 'Nickname', 'Location', 'Founded',
                                    'Joined', 'Left', 'Enrollment', 'Conference'])

# append rows to df
for df in df_list:
    minor_conf_df = minor_conf_df.append(df, ignore_index=True)

len(minor_conf_df)

minor_conf_df.sample(20)



C:\Users\Justin\AppData\Local\Temp\ipykernel_3236\3667365479.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  minor_conf_df = minor_conf_df.append(df, ignore_index=True)


,Institution,Nickname,Location,Founded,Joined,Left,Enrollment,Conference
162,Hofstra University,Pride,"Hempstead, New York",1935,2007,2009.0,10871,CAA
562,Thiel College,Tomcats,"Greenville, Pennsylvania",1866,1958,NaN,790.0,Presidents Athletic Conf (PAC)
281,University of Indianapolis,Greyhounds,"Indianapolis, Indiana",1902,1970,1978,5500,Indiana Collegiate Conference
472,Pacific (CA),Tigers,"Stockton, CA",1911,1925,1942,,Far Western
467,Cal State Hayward,Pioneers,"Hayward, CA",1956,1961,1998,,Far Western
311,Monmouth College,Fighting Scots,"Monmouth, Illinois",1853,1908,1936,1300.0,IIAC
617,California - Santa Barbara,Gauchos,Santa Barbara,1891,1931,1937,22850,Southern California Intercollegiate Athletic
629,Louisiana Tech University,BulldogsLady Techsters,"Ruston, Louisiana",1894,1971,1987,NaN,Southland
351,Dallas Baptist University,Patriots,"Dallas, Texas",1898,2019[c],NaN,5500,Lone Star
653,Texas Tech University,Red Raiders,"Lubbock, Texas",1923,1925,1925,40000,Texas Intercollegiate Athletic


In [45]:
## SAVE CSV TO TEMP FOLDER
minor_conf_df.to_csv('../data/conference/minor_conf_df.csv', index=False)